In [1]:
# -*- coding: utf-8 -*-
"""
Production runner with Historical vs Closed-Loop modes.

What it does (idempotent):
  1) Ensures ML predictions for the 6-month lookback + today's stamps → CSV cache
  2) Ensures GP residual rows for the 6-month window → Pickle cache
  3) Fits GP on the 6-month slice
  4) (Per stamp) Auto-tunes per-product α by slope fidelity (rc0-injected), saves fidelity
  5) Runs multi-knob RCOT optimizer (with anchored objective), prints/saves RCOT moves, price audit
  6) Saves curves (overrides-only α, rc0-injected) + per-geometry fidelity audit
  7) At the end: builds a counterfactual schedule and simulates corrected yields + margin
     (consistent end-of-run check; for online closed-loop the state is updated in-loop)

Supports modes:
  - historical
  - closed_loop: apply recommended RCOTs on a stateful copy of X and hold until next decision.
    Keeps caches separate via a cache_tag to avoid polluting historical ones.
"""

from __future__ import annotations
import json
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Sequence, List, Optional, Tuple, Any
import numpy as np
import pandas as pd
from src.ml_predictor import MLPredictor, MLPredictorConfig

import warnings
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names, but GaussianProcessRegressor was fitted with feature names",
    category=UserWarning,
    module="sklearn"
)

# ---- project imports ----
from importlib import reload
import src.gp_residuals as gpmod
import src.optimizer    as opt


# =================== CONFIG (static) ===================
LOOKBACK_6M    = pd.Timedelta(days=180)
MIN_TR_ROWS    = 180
GP_JOBS        = 8

START          = pd.Timestamp('2025-01-01')
END            = pd.Timestamp('2025-05-19')

OUT_DIR        = Path("prod_out")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Base OUT dir; mode-specific subdirs will be derived
OUT_DIR_BASE   = Path("prod_out")
OUT_DIR_BASE.mkdir(parents=True, exist_ok=True)

# RCOT bounds (single source of truth)
RC_BOUNDS = {
    'LF_NAPH':     (830.0, 853.0),
    'GF_GAS':      (860.0, 890.0),
    'GF_HYB_NAPH': (830.0, 853.0),
}

# canonical products
PRODS_CANON = ("Ethylene","Propylene","Mixed C4","RPG","Hydrogen","Tail Gas")
# canonical internal keys for corrected_yields_for_row
PRODS_INTERNAL = ['Ethylene','Propylene','MixedC4','RPG','Ethane','Propane','Hydrogen','Tail_Gas']
TARGET_COLS = [f'{p}_prod_t+1' for p in PRODS_INTERNAL]

# =================== HELPERS ===================
def _ts_per_day(index: pd.DatetimeIndex, day: pd.Timestamp) -> List[pd.Timestamp]:
    d0 = pd.Timestamp(day).normalize()
    d1 = d0 + pd.Timedelta(days=1)
    idx = index[(index >= d0) & (index < d1)]
    return list(idx)

def _safe_merge_csv(path: Path, df_new: pd.DataFrame, key: str | list[str] = 'timestamp') -> pd.DataFrame:
    key_cols = [key] if isinstance(key, str) else list(key)
    if path.exists():
        old = pd.read_csv(path, parse_dates=[k for k in key_cols if 'time' in k.lower()])
        old = old.set_index(key_cols).sort_index()
    else:
        if len(key_cols) == 1:
            k = key_cols[0]
            idx = (pd.DatetimeIndex([], name=k) if 'time' in k.lower() else pd.Index([], name=k))
            old = pd.DataFrame(index=idx)
        else:
            idx = pd.MultiIndex.from_arrays([[] for _ in key_cols], names=key_cols)
            old = pd.DataFrame(index=idx)

    new = df_new.copy()
    for k in key_cols:
        if k in new.columns and 'time' in k.lower():
            new[k] = pd.to_datetime(new[k], errors='coerce')
    new = new.set_index(key_cols).sort_index()

    out = pd.concat([old[~old.index.isin(new.index)], new], axis=0).sort_index()
    out_reset = out.reset_index()
    out_reset.to_csv(path, index=False)
    return out_reset

def _safe_merge_pickle(path: Path, df_new: pd.DataFrame, min_index: pd.Timestamp | None = None) -> pd.DataFrame:
    if path.exists():
        old = pd.read_pickle(path)
        old.index = pd.to_datetime(old.index)
    else:
        old = pd.DataFrame()

    if df_new is not None and len(df_new):
        df_new = df_new.copy()
        df_new.index = pd.to_datetime(df_new.index)
        out = pd.concat([old, df_new], axis=0)
    else:
        out = old

    if min_index is not None and not out.empty:
        out = out.loc[out.index >= pd.Timestamp(min_index)]

    out = out[~out.index.duplicated(keep='last')].sort_index()
    out.to_pickle(path)
    return out

def realized_margin_from_Y(ts: pd.Timestamp, x_row: pd.Series, Y_12h: pd.DataFrame, margin_fn) -> float:
    ydict = {}
    for p in PRODS_INTERNAL:
        col = f"{p}_prod_t+1"
        ydict[col] = float(Y_12h.at[ts, col]) if (ts in Y_12h.index and col in Y_12h.columns) else 0.0
    return float(margin_fn(ts, x_row, ydict))

def _geometry_label_for_row(row: pd.Series) -> str:
    n = sum(row.get(f'Naphtha_chamber{i}', 0.0) for i in range(1,7))
    g = sum(row.get(f'Gas Feed_chamber{i}', 0.0) for i in (4,5,6))
    if n>0 and g>0: return 'GF_HYB_NAPH'
    if n>0:         return 'LF_NAPH'
    if g>0:         return 'GF_GAS'
    return 'NONE'

def _norm_geom(g: str) -> str:
    return str(g).strip().replace(" ","_").upper()

def _bounds_for_geoms() -> dict[str, tuple[float,float]]:
    return {'LF_NAPH':(800.0,895.0), 'GF_GAS':(820.0,910.0), 'GF_HYB_NAPH':(800.0,895.0)}

def _rc_grid_for(ts_row: pd.Series, geom: str, lo: float, hi: float, points: int=15) -> np.ndarray:
    rc0 = gpmod.rc0_guess_for_geom(ts_row, geom, fallback_rc=None)
    base = np.linspace(lo, hi, points)
    return np.unique(np.r_[base, rc0]) if rc0 is not None else base

# =================== ML CACHE ADAPTER ===================
class MLCacheAdapter:
    """Adapter for cached ML predictions; provides transform() (no-op) and predict_row()."""
    def __init__(self, pred_df: pd.DataFrame):
        self.pred_df = pred_df

    def transform(self, X_12h: pd.DataFrame, Y_12h: pd.DataFrame):
        # interface compliance; not used by GP table
        return X_12h

    def predict_row(self, row_like, **kwargs):
        ts = getattr(row_like, 'name', None)
        if ts is None or ts not in self.pred_df.index:
            return {}
        s = self.pred_df.loc[ts]
        return {c: float(s.get(c, np.nan)) for c in self.pred_df.columns}

# =================== FIDELITY (slope-only gate) ===================
def _robust_slope_metrics(curve: pd.DataFrame, prod: str) -> tuple[float,float]:
    if curve is None or curve.empty or len(curve) < 3:
        return (np.nan, 0.0)
    s = curve.get(f'{prod}_SRTO_tph'); c = curve.get(f'{prod}_CORR_tph')
    if s is None or c is None: return (np.nan, 0.0)
    s = s.to_numpy(float); c = c.to_numpy(float)
    ds, dc = np.diff(s), np.diff(c)
    ds_f = ds[np.isfinite(ds)]
    if ds_f.size == 0: return (np.nan, 0.0)
    eps  = 0.01*(np.nanpercentile(np.abs(ds_f),95) + 1e-12)
    mask = np.abs(ds) >= eps
    if mask.sum() < 3 or not np.isfinite(dc[mask]).all(): return (np.nan, float(mask.mean()))
    return (float(np.corrcoef(ds[mask], dc[mask])[0,1]), float(mask.mean()))

def _rcot_setter_single_knob(knob: str):
    def _setter(row_like: pd.Series, rc: float) -> pd.Series:
        row_like[knob] = float(rc)
        return row_like
    return _setter

def _local_rc_grid(rc0: float, lo: float, hi: float, halfspan: float = 10.0, n: int = 9) -> np.ndarray:
    if not np.isfinite(rc0):
        return np.linspace(lo, hi, n)
    a, b = max(lo, rc0 - halfspan), min(hi, rc0 + halfspan)
    return np.unique(np.r_[np.linspace(a, b, n), rc0])

def _knob_from_leg(leg: dict) -> str:
    ch = leg['chamber']
    if leg['feed'] == 'gas':
        return f'RCOT_gas_chamber{ch}'
    if ch in (1,2,3):
        return f'RCOT_chamber{ch}'
    return f'RCOT_naphtha_chamber{ch}'

def build_knob_fidelity_gate(*, ts, row_current, gp, X_12h, merged_lims, pipeline, ml_cached,
                             rc_bounds_map, thr_corr=0.92, min_cov=0.20,
                             halfspan_ok=10.0, halfspan_fallback=2.0):
    """
    Returns: (bounds_by_knob: dict, knob_fid_df: DataFrame)
      PASS → trust ±halfspan_ok (clipped to geometry bounds)
      FAIL → try SRTO finite-difference fallback; if OK → ±halfspan_fallback; else FREEZE (rc0,rc0)
    """
    comp_row = gp._comp_row_for_ts(merged_lims, ts)
    legs = pipeline._chamber_legs_from_state(row_current, feed_thr=1.0)  # active legs only

    rows, bounds_by_knob = [], {}
    KEY = ['Ethylene','Propylene','RPG']  # key revenue drivers

    for leg in legs:
        geom = leg['geometry']  # 'LF_NAPH' | 'GF_GAS' | 'GF_HYB_NAPH'
        knob = _knob_from_leg(leg)
        rc0  = float(row_current.get(knob, np.nan))
        lo, hi = rc_bounds_map.get(geom, (800.0, 895.0))
        rc_grid = _local_rc_grid(rc0, lo, hi, halfspan=halfspan_ok, n=9)

        # Local anchored curve with tuned α (already set on gp)
        curve, _ = gpmod.anchored_curve_at_ts(
            gp=gp, X_12h=X_12h, merged_lims=merged_lims, pipeline=pipeline, ml=ml_cached,
            ts=ts, rcot_setter=_rcot_setter_single_knob(knob), rc_grid=rc_grid,
            use_gp_delta=True, alpha=0.0
        )

        flags = []
        for p in KEY:
            sc, cov = _robust_slope_metrics(curve, p)
            rows.append({'timestamp': ts, 'geometry': geom, 'chamber': leg['chamber'],
                         'knob': knob, 'product': p, 'slope_corr': sc, 'sign_cov': cov})
            flags.append((sc >= thr_corr) and (cov >= min_cov))

        if all(flags):
            a = max(lo, rc0 - halfspan_ok); b = min(hi, rc0 + halfspan_ok)
            bounds_by_knob[knob] = (a, b)
            continue

        # Fallback: SRTO central difference at rc0 ± 5°C
        try:
            dC = 5.0
            def _spot(rc):
                r = row_current.copy(); r[knob] = float(np.clip(rc, lo, hi))
                spot = pipeline.predict_spot_plant(r, comp_row, feed_thr=0.1)
                return spot['totals_tph'] if spot.get('status') == 'ok' else {}
            y_lo = _spot(rc0 - dC); y_hi = _spot(rc0 + dC)
            fd_ok = all(abs(y_hi.get(k,0.0) - y_lo.get(k,0.0)) > 1e-6 for k in KEY)
        except Exception:
            fd_ok = False

        if fd_ok:
            a = max(lo, rc0 - halfspan_fallback); b = min(hi, rc0 + halfspan_fallback)
            bounds_by_knob[knob] = (a, b)
        else:
            bounds_by_knob[knob] = (rc0, rc0)

    return bounds_by_knob, pd.DataFrame(rows)


def auto_alpha_until_pass_for_ts(
    gp: gpmod.GPResiduals,
    ts: pd.Timestamp,
    row0: pd.Series,
    X_12h: pd.DataFrame,
    merged_lims: pd.DataFrame,
    pipeline,
    ml_cached,
    thr_corr: float=0.92,
    min_cov: float=0.20,
    rc_points: int=15
) -> tuple[dict, pd.DataFrame, pd.DataFrame]:

    geoms = []
    n = sum(float(row0.get(f'Naphtha_chamber{i}',0.0)) for i in range(1,7))
    g = sum(float(row0.get(f'Gas Feed_chamber{i}',0.0)) for i in (4,5,6))
    if n > 1.0: geoms.append('LF_NAPH')
    if g > 1.0: geoms.append('GF_GAS')
    if n > 1.0 and g > 1.0: geoms.append('GF_HYB_NAPH')

    setter_map = {'LF_NAPH':gpmod.rcot_setter_lf_naph, 'GF_GAS':gpmod.rcot_setter_gf_gas, 'GF_HYB_NAPH':gpmod.rcot_setter_hybrid}
    rc_bounds  = _bounds_for_geoms()
    setter_map = {g:setter_map[g] for g in geoms}; rc_bounds = {g:rc_bounds[g] for g in geoms}

    overrides: dict[tuple[str,str], float] = {}
    alpha_grid = np.r_[np.linspace(0.35, 0.0, 8), 0.0]

    # First pass: find failing pairs
    gp.set_alpha_overrides({})
    fid_rows = []
    for g in geoms:
        lo, hi = rc_bounds[g]; rc_grid = _rc_grid_for(row0, g, lo, hi, points=rc_points)
        curve, _ = gpmod.anchored_curve_at_ts(
            gp=gp, X_12h=X_12h, merged_lims=merged_lims, pipeline=pipeline,
            ml=ml_cached, ts=ts, rcot_setter=setter_map[g], rc_grid=rc_grid,
            use_gp_delta=True, alpha=0.0
        )
        for p in gpmod.PRODUCTS:
            sc, cov = _robust_slope_metrics(curve, p)
            fid_rows.append({'timestamp': ts, 'geometry': _norm_geom(g), 'product': p, 'slope_corr': sc, 'sign_cov': cov})
    fid0 = pd.DataFrame(fid_rows)
    fails = fid0.loc[(fid0['slope_corr'] < thr_corr) | (fid0['sign_cov'] < min_cov), ['product','geometry']].drop_duplicates()

    # Iterate α for failing pairs
    for prod, geom in fails.itertuples(index=False):
        tcol = gpmod.TARGET_MAP[prod]
        best = None
        for a in alpha_grid:
            trial = {**overrides, (_norm_geom(geom), tcol): float(a)}
            gp.set_alpha_overrides(trial)
            lo, hi = rc_bounds[geom]; rc_grid = _rc_grid_for(row0, geom, lo, hi, points=rc_points)
            curve, _ = gpmod.anchored_curve_at_ts(
                gp=gp, X_12h=X_12h, merged_lims=merged_lims, pipeline=pipeline,
                ml=ml_cached, ts=ts, rcot_setter=setter_map[geom], rc_grid=rc_grid,
                use_gp_delta=True, alpha=0.0
            )
            sc, cov = _robust_slope_metrics(curve, prod)
            if sc >= thr_corr and cov >= min_cov:
                best = float(a); break
        overrides[(_norm_geom(geom), tcol)] = (0.0 if best is None else best)

    # Final detail/summary with overrides
    gp.set_alpha_overrides(overrides)
    fid_rows = []
    for g in geoms:
        lo, hi = rc_bounds[g]; rc_grid = _rc_grid_for(row0, g, lo, hi, points=rc_points)
        curve, _ = gpmod.anchored_curve_at_ts(
            gp=gp, X_12h=X_12h, merged_lims=merged_lims, pipeline=pipeline,
            ml=ml_cached, ts=ts, rcot_setter=setter_map[g], rc_grid=rc_grid,
            use_gp_delta=True, alpha=0.0
        )
        for p in gpmod.PRODUCTS:
            sc, cov = _robust_slope_metrics(curve, p)
            a_col, c_col = f'{p}_ANCHOR_tph', f'{p}_CORR_tph'
            diff = (curve[c_col].astype(float) - curve[a_col].astype(float)).abs().to_numpy(float) if (a_col in curve and c_col in curve) else np.array([np.nan])
            am_min = float(np.nanmin(diff))
            fid_rows.append({'timestamp': ts, 'geometry': _norm_geom(g), 'product': p,
                             'slope_corr': sc, 'sign_cov': cov, 'anchor_miss_min': am_min})
    fid = pd.DataFrame(fid_rows)
    fid['slope_ok'] = (fid['slope_corr'] >= thr_corr) & (fid['sign_cov'] >= min_cov)
    summ = (fid.groupby(['product','geometry'])['slope_ok']
                .agg(pct_ok=lambda s: float(100.0*s.mean()), n='count')
                .reset_index()
                .sort_values(['product','geometry']))
    return overrides, fid, summ

# =================== Counterfactual schedule & simulate ===================
def build_rcot_schedule_from_recs(rec_df: pd.DataFrame) -> list[tuple[pd.Timestamp, dict]]:
    if 'timestamp' in rec_df.columns:
        df = rec_df.set_index('timestamp')
    else:
        df = rec_df.copy()
    df = df.sort_index()
    knobs_cols = [c for c in df.columns if c.startswith('rcot_opt_')]
    sched = []
    for ts, r in df.iterrows():
        setpoints = {}
        for c in knobs_cols:
            v = r.get(c, np.nan)
            if pd.notna(v):
                knob = c.replace('rcot_opt_', '')
                setpoints[knob] = float(v)
        if setpoints:
            sched.append((pd.Timestamp(ts), setpoints))
    return sched

def apply_schedule_to_X(X_12h: pd.DataFrame, schedule: list[tuple[pd.Timestamp, dict]],
                        start: pd.Timestamp | None = None,
                        end:   pd.Timestamp | None = None,
                        hold: str = "hold_until_next") -> pd.DataFrame:
    X_sim = X_12h.copy(); idx = X_sim.index
    if start is None: start = idx.min()
    if end   is None: end   = idx.max()
    schedule = sorted([(pd.Timestamp(t), d) for (t,d) in schedule if start <= pd.Timestamp(t) <= end],
                      key=lambda x: x[0])
    if not schedule:
        return X_sim
    schedule2 = schedule + [(pd.Timestamp(end) + pd.Timedelta(seconds=1), {})]
    for (t0, setpoints), (t1, _) in zip(schedule2[:-1], schedule2[1:]):
        mask = (idx >= t0) & (idx < t1)
        if not mask.any(): continue
        for knob, val in setpoints.items():
            if knob in X_sim.columns:
                X_sim.loc[mask, knob] = float(val)
    return X_sim

def simulate_path_corrected(X_sim: pd.DataFrame,
                            merged_lims: pd.DataFrame,
                            pipeline,
                            gp: gpmod.GPResiduals,
                            gps_dict: dict,
                            feature_cols_gp: list[str],
                            price_provider: opt.PriceProvider,
                            total_spyro_yield_for_now,
                            fg_consts: opt.FuelGasConstants,
                            alpha_overrides: dict | None = None,
                            start: pd.Timestamp | None = None,
                            end:   pd.Timestamp | None = None) -> tuple[pd.DataFrame, pd.DataFrame]:
    idx = X_sim.index
    if start is None: start = idx.min()
    if end   is None: end   = idx.max()
    stamps = idx[(idx >= start) & (idx <= end)]

    margin_fn = opt.make_margin_fn_excel_delta(price_provider=price_provider,
                                               total_spyro_yield_for_now=total_spyro_yield_for_now,
                                               spyro_ctx=None, fg_constants=fg_consts)

    Y_rows, M_rows = [], []
    for ts in stamps:
        row = X_sim.loc[ts].copy(); row.name = ts
        y   = opt.corrected_yields_for_row(row, gps_dict, feature_cols_gp,
                                           total_spyro_yield_for_now, spyro_ctx=None,
                                           alpha_overrides=alpha_overrides)
        Y_rows.append({'timestamp': ts, **y})
        M_rows.append({'timestamp': ts, 'margin_per_h': float(margin_fn(ts, row, y))})

    Y_sim = pd.DataFrame(Y_rows).set_index('timestamp').sort_index()
    M_sim = pd.DataFrame(M_rows).set_index('timestamp').sort_index()
    return Y_sim, M_sim

def _cache_path(prefix: str, cache_tag: str, mode: str, train_mode: str, ext: str) -> Path:
    tag = cache_tag or ""
    return OUT_DIR / f"{prefix}{tag}_{mode}_{train_mode}.{ext}"

def effective_target_cols(Y_12h: pd.DataFrame, explicit: Optional[Sequence[str]] = None) -> List[str]:
    from_cols = [c for c in Y_12h.columns if c.endswith("_prod_t+1")]
    if explicit is None:
        return sorted(set(from_cols))
    return sorted(set(explicit) | set(from_cols))

def seed_sim_state(Y_12h: pd.DataFrame, tcols: Sequence[str], seed_until: pd.Timestamp) -> pd.DataFrame:
    ys = pd.DataFrame(index=Y_12h.index, columns=list(tcols), dtype=float)
    common = [c for c in tcols if c in Y_12h.columns]
    ys.loc[:, common] = Y_12h[common]
    ys.loc[ys.index > seed_until, :] = np.nan
    return ys

def ensure_ml_preds_for(
    stamps: Sequence[pd.Timestamp],
    Xsrc: pd.DataFrame,
    Ysrc: pd.DataFrame,
    lookback: pd.Timedelta,
    target_cols: Sequence[str],
    *,
    mode: str,                 # 'historical' | 'closed_loop'
    train_mode: str,           # 'historical' | 'simulated'
    cache_tag: str = "",
    Y_sim_state: Optional[pd.DataFrame] = None,
) -> pd.DataFrame:
    """
    Predict ML targets for `stamps`, keeping a cache per (mode, train_mode, cache_tag).
    - train_mode=historical  → fit on Ysrc (ground truth)
    - train_mode=simulated   → fit on Y_sim_state (simulated truth)
    - Lags always read from (Y_sim_state ⊕ Ysrc) during closed-loop to avoid KeyErrors.
    """
    cache_csv = _cache_path("ml_cache", cache_tag, mode, train_mode, "csv")
    if cache_csv.exists():
        pred_cache = pd.read_csv(cache_csv, parse_dates=["timestamp"]).set_index("timestamp")
    else:
        pred_cache = pd.DataFrame(index=pd.DatetimeIndex([], name="timestamp"))

    missing = sorted(ts for ts in stamps if ts not in pred_cache.index)
    if not missing:
        return pred_cache

    rows = []
    tcols = list(target_cols)

    for ts in missing:
        ts_start = ts - lookback
        tr = Xsrc.index[(Xsrc.index >= ts_start) & (Xsrc.index < ts)]
        if len(tr) < MIN_TR_ROWS:
            # not enough history to fit
            rows.append({"timestamp": ts, **{c: np.nan for c in tcols}})
            continue

        Y_train = (Y_sim_state if (train_mode == "simulated" and Y_sim_state is not None) else Ysrc)

        ml = MLPredictor(
            target_cols=tcols,
            cfg=MLPredictorConfig(
                ds_prefixes=["DS_chamber"],
                add_virtual_rcots=True,
                build_lag1_from_targets=True,
                lgbm_params=dict(verbosity=-1, n_jobs=2),
            ),
        ).fit(Xsrc.loc[tr], Y_train.loc[tr])

        # choose lag source (combine so missing columns are tolerated)
        if mode == "closed_loop" and Y_sim_state is not None:
            Y_lag_src = Y_sim_state.combine_first(Ysrc)
        else:
            Y_lag_src = Ysrc

        pm = ml.predict_row(Xsrc.loc[ts], Y_for_lags=Y_lag_src)
        rows.append({"timestamp": ts, **{c: float(pm.get(c, np.nan)) for c in tcols}})

    if rows:
        add_df = pd.DataFrame(rows).set_index("timestamp")
        pred_cache = pd.concat([pred_cache, add_df], axis=0).sort_index()
        pred_cache.to_csv(cache_csv, index=True)

    return pred_cache

def _legs_to_df(legs: list[dict]) -> pd.DataFrame:
    rows = []
    keep = ['Ethylene','Propylene','MixedC4','RPG','PFO','Ethane','Propane','Tail_Gas','Hydrogen']
    for lg in legs:
        rec = {
            'chamber': lg.get('chamber'),
            'feed': lg.get('feed'),
            'geometry': lg.get('geometry'),
            'rcot_used': float(lg.get('rcot_used', np.nan)),
            'feed_tph': float(lg.get('feed_tph', 0.0)),
            'IRET': float(lg.get('IRET', np.nan))
        }
        for k in keep:
            rec[f'{k}_tph'] = float(lg.get('tph', {}).get(k, 0.0))
        rows.append(rec)
    return pd.DataFrame(rows)

def _ch_from_knob(knob: str) -> int | None:
    if knob.startswith('RCOT_chamber'):            return int(knob.replace('RCOT_chamber',''))
    if knob.startswith('RCOT_naphtha_chamber'):    return int(knob.replace('RCOT_naphtha_chamber',''))
    if knob.startswith('RCOT_gas_chamber'):        return int(knob.replace('RCOT_gas_chamber',''))
    return None

# =================== RUNNER ===================
def run_production(X_12h: pd.DataFrame, Y_12h: pd.DataFrame,
                   merged_lims: pd.DataFrame, pipeline,
                   prices_df: pd.DataFrame,
                   total_spyro_yield_for_now,
                   start: pd.Timestamp,
                   end:   pd.Timestamp | None = None,
                   mode: str = 'historical',
                   closed_loop_opts: dict | None = None):
    """
    mode:
      - 'historical': baseline behavior
      - 'closed_loop': step & hold recommended RCOTs on a stateful copy of X;
                       uses simulated lags for ML/inference; separate caches via cache_tag.
    """
    assert mode in ('historical','closed_loop')
    closed_loop_opts = closed_loop_opts or {}
    apply_timing  = closed_loop_opts.get('apply_timing', 'next_day')
    hold_policy   = closed_loop_opts.get('hold_policy',  'hold_until_next')
    ml_train_mode = closed_loop_opts.get('ml_train_mode','historical')
    gp_train_mode = closed_loop_opts.get('gp_train_mode','historical')
    cache_tag     = closed_loop_opts.get('cache_tag',    '' if mode=='historical' else '_sim')

    # Out dirs & caches
    OUT_DIR = OUT_DIR_BASE / (('closed_loop' + cache_tag) if mode=='closed_loop' else 'historical')
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    CURVES_DIR = OUT_DIR / "curves"; CURVES_DIR.mkdir(parents=True, exist_ok=True)
    FID_DIR    = OUT_DIR / "fidelity"; FID_DIR.mkdir(parents=True, exist_ok=True)
    MOV_DIR    = OUT_DIR / "rcot_moves"; MOV_DIR.mkdir(parents=True, exist_ok=True)
    AK_DIR     = OUT_DIR / "active_knobs"; AK_DIR.mkdir(parents=True, exist_ok=True)
    AUD_DIR    = OUT_DIR / "audits"; AUD_DIR.mkdir(parents=True, exist_ok=True)
    SNAP_DIR   = OUT_DIR / "multi_snapshots"; SNAP_DIR.mkdir(parents=True, exist_ok=True)
    CF_DIR     = OUT_DIR / "counterfactual"; CF_DIR.mkdir(parents=True, exist_ok=True)
    KNOB_FID_DIR = FID_DIR / "knob"; KNOB_FID_DIR.mkdir(parents=True, exist_ok=True)
    ATTR_DIR   = OUT_DIR / "attribution"; ATTR_DIR.mkdir(parents=True, exist_ok=True)

    pp = opt.PriceProvider(prices_df)
    gpmod.set_rcot_groups_from_columns(X_12h.columns)

    # fuel-gas constants
    fg_consts = opt.FuelGasConstants(
        rc_ref_naph=840.0, rc_ref_gas=880.0,
        cp_wavg_kcal_per_ton_K=411.488209,
        dH_eth_kcal_per_ton=1_080_970.0,
        dH_prop_kcal_per_ton=673_409.0,
        dH_fg_kcal_per_ton=926_147.0,
        fuel_gas_heat_content_kcal_per_ton=15_294_088.0
    )

    idx_all = X_12h.index.sort_values()
    if end is None: end = idx_all.max().normalize()

    # Closed-loop state
    X_state = X_12h.copy()
    tcols_for_sim = [c for c in TARGET_COLS if c in Y_12h.columns]
    Y_sim_state = (Y_12h.reindex(columns=tcols_for_sim).copy() if mode=='closed_loop' else None)

    # ---------- GP training cache helper ----------
    def ensure_gp_train_for_window(train_start: pd.Timestamp, train_end: pd.Timestamp,
                                   Xs: pd.DataFrame, Ys: pd.DataFrame,
                                   merged_lims: pd.DataFrame, pipeline,
                                   ml_cached: MLCacheAdapter) -> pd.DataFrame:
        if (OUT_DIR / f"gp_train_cache{cache_tag}.pkl").exists():
            gp_cache = pd.read_pickle(OUT_DIR / f"gp_train_cache{cache_tag}.pkl")
            gp_cache.index = pd.to_datetime(gp_cache.index)
            cached_idx = gp_cache.index
        else:
            gp_cache = pd.DataFrame(); cached_idx = pd.DatetimeIndex([])

        idx_in_win = Xs.index[(Xs.index >= train_start) & (Xs.index <= train_end)]
        need = [ts for ts in idx_in_win if ts not in cached_idx]
        if need:
            s = min(need); e = max(need)
            Xsrc = (X_state if (mode=='closed_loop' and gp_train_mode=='simulated') else Xs)
            Ysrc = (Y_sim_state if (mode=='closed_loop' and Y_sim_state is not None and gp_train_mode=='simulated') else Ys)
            df_new = gpmod.GPResiduals.build_training_table(
                Xsrc, Ysrc, merged_lims, pipeline,
                start=s, end=e, feed_thr=0.1,
                feature_cfg=gpmod.GPFeatureConfig(
                    ds_prefixes=['DS_chamber'], rcot_prefixes=['RCOT_'],
                    feed_prefixes=['Naphtha_chamber','Gas Feed_chamber'],
                    include_ratio_naphtha=True, include_geometry_flags=True
                ),
                residual_kind='ml', ml=ml_cached
            )
            if not df_new.empty:
                df_new.index = pd.to_datetime(df_new.index)
                _safe_merge_pickle(OUT_DIR / f"gp_train_cache{cache_tag}.pkl", df_new, min_index=train_start)
        if (OUT_DIR / f"gp_train_cache{cache_tag}.pkl").exists():
            win = pd.read_pickle(OUT_DIR / f"gp_train_cache{cache_tag}.pkl")
            win.index = pd.to_datetime(win.index)
            win = win.loc[(win.index >= train_start) & (win.index <= train_end)].sort_index()
        else:
            win = pd.DataFrame()
        return win

    # ===== daily loop =====
    for day in pd.date_range(pd.Timestamp(start).normalize(), pd.Timestamp(end).normalize(), freq='D'):
        stamps = _ts_per_day(idx_all, day)
        if not stamps:
            continue
        te = stamps[-1] - pd.Timedelta(hours=12)
        ts_train_start = te - LOOKBACK_6M

        # ---- ML cache for the training window (respect provenance) ----
        tcols_ml = [c for c in TARGET_COLS if c in Y_12h.columns]
        Xsrc_train = X_state if (mode=='closed_loop' and ml_train_mode=='simulated') else X_12h
        train_stamps = list(X_12h.index[(X_12h.index >= ts_train_start) & (X_12h.index <= te)])
        pred_cache_train = ensure_ml_preds_for(
            train_stamps, Xsrc_train, Y_12h, lookback=LOOKBACK_6M, target_cols=tcols_ml,
            mode=mode, train_mode=ml_train_mode, cache_tag=cache_tag,
            Y_sim_state=(Y_sim_state if mode == 'closed_loop' else None),
        )
        ml_cached_train = MLCacheAdapter(pred_cache_train)

        # ---- GP train cache ----
        df_train_win = ensure_gp_train_for_window(ts_train_start, te, X_12h, Y_12h, merged_lims, pipeline, ml_cached_train)
        if df_train_win.empty:
            continue

        gp = gpmod.GPResiduals(
            feature_cfg=gpmod.GPFeatureConfig(
                ds_prefixes=['DS_chamber'], rcot_prefixes=['RCOT_'],
                feed_prefixes=['Naphtha_chamber','Gas Feed_chamber'],
                include_ratio_naphtha=True, include_geometry_flags=True
            ),
            n_restarts=2, normalize_y=True
        ).fit_parallel(df_train_win, n_jobs=GP_JOBS)

        gps_dict = {f'{p}_prod_t+1': gp.models_[p] for p in gpmod.PRODUCTS if p in gp.models_}
        feature_cols_gp = gp.feature_names_

        rec_rows = []
        for ts in stamps:
            # ---- current row from state ----
            row_current = (X_state if mode=='closed_loop' else X_12h).loc[ts].copy(); row_current.name = ts

            # ensure Spyro sees a timestamp
            def _spyro_ts(row_like, short_key, ctx=None, _ts=ts):
                r = row_like
                if getattr(r, 'name', None) is None:
                    r = r.copy(); r.name = _ts
                return total_spyro_yield_for_now(r, short_key, ctx)

            geom = _geometry_label_for_row(row_current)
            naph_b, gas_b = RC_BOUNDS.get('LF_NAPH'), RC_BOUNDS.get('GF_GAS')

            margin_fn = opt.make_margin_fn_excel_delta(
                price_provider=pp,
                total_spyro_yield_for_now=_spyro_ts,
                spyro_ctx=None,
                fg_constants=fg_consts
            )

            # ---- spot ML baseline using UPDATED state ----
            pred_cache_spot = ensure_ml_preds_for(
                [ts], (X_state if mode=='closed_loop' else X_12h), Y_12h,
                lookback=LOOKBACK_6M, target_cols=tcols_ml,
                mode=mode, train_mode=ml_train_mode, cache_tag=cache_tag,
                Y_sim_state=(Y_sim_state if mode == 'closed_loop' else None),
            )
            ml_cached = MLCacheAdapter(pred_cache_spot)
            y0 = ml_cached.predict_row(row_current)
            m0 = margin_fn(ts, row_current, y0)
            m_real = realized_margin_from_Y(ts, row_current, Y_12h, margin_fn)

            # ---- fidelity + knob gate ----
            overrides, fid_detail, fid_summary = auto_alpha_until_pass_for_ts(
                gp=gp, ts=ts, row0=row_current, X_12h=X_12h,
                merged_lims=merged_lims, pipeline=pipeline, ml_cached=ml_cached,
                thr_corr=0.92, min_cov=0.20, rc_points=15
            )
            FID_DIR.mkdir(exist_ok=True, parents=True)
            fid_detail.to_csv((OUT_DIR / "fidelity" / f"fidelity_detail_{ts:%Y%m%d_%H%M}.csv"), index=False)
            fid_summary.to_csv((OUT_DIR / "fidelity" / f"fidelity_summary_{ts:%Y%m%d_%H%M}.csv"), index=False)

            bounds_by_knob, knob_fid = build_knob_fidelity_gate(
                ts=ts, row_current=row_current, gp=gp,
                X_12h=X_12h, merged_lims=merged_lims, pipeline=pipeline, ml_cached=ml_cached,
                rc_bounds_map=RC_BOUNDS, thr_corr=0.92, min_cov=0.20,
                halfspan_ok=10.0, halfspan_fallback=2.0
            )
            (OUT_DIR / "fidelity" / "knob").mkdir(exist_ok=True, parents=True)
            knob_fid.to_csv(OUT_DIR / "fidelity" / "knob" / f"fidelity_knob_{ts:%Y%m%d_%H%M}.csv", index=False)

            # ---- Optimize (anchored objective) ----
            res = opt.optimize_rcot_for_ts_multi(
                ts=ts, row0=row_current,
                gps=gps_dict, feature_cols_gp=feature_cols_gp,
                total_spyro_yield_for_now=total_spyro_yield_for_now, spyro_ctx=None,
                price_provider=pp,
                objective='per_hour',
                naph_bounds=naph_b, gas_bounds=gas_b,
                trust_delta_C=5.0,                     # your ±5 °C trust window
                use_recycle_fixed_point=False, recycle_fn=None,
                margin_mode='excel_delta', fg_constants=fg_consts,
                alpha_overrides=None,
                enable_de=True, enable_slsqp=True,
                bounds_by_knob=bounds_by_knob,
                anchored_from_ml=True,                 # ← anchored mode
                gp=gp, pipeline=pipeline, merged_lims=merged_lims, ml_cached=ml_cached,
                alpha_default_for_anchor=0.0
            )

            row_opt = res['row_opt']; y_opt = res['yields_opt']
            m_base  = res['margin_current_per_h']; m_opt = res['margin_opt_per_h']; d_m = res['improvement_per_h']

            print("\n=== MULTI-KNOB RESULT ===")
            print("Status:", res.get('status'))
            print("ΔMargin $/h:", f"{d_m:,.2f}")
            print("RCOT* (per chamber):")
            for k,v in res['rcot_opt'].items():
                cur = float(row_current.get(k, np.nan))
                print(f"  {k:24s}  {cur:7.2f} → {float(v):7.2f}  (Δ {float(v)-cur:+.2f})")

            # ---- Flat record ----
            flat = {
                'timestamp': ts, 'geometry': geom, 'status': res['status'],
                'margin_baseline_per_h': float(m0),
                'margin_opt_per_h':      float(m_opt),
                'improvement_per_h':     float(d_m),
                'margin_realized_per_h': float(m_real),
                'margin_current_per_h':  float(m_base),
            }
            rcot_cols = [f'RCOT_chamber{i}' for i in (1,2,3)] \
                      + [f'RCOT_naphtha_chamber{i}' for i in (4,5,6)] \
                      + [f'RCOT_gas_chamber{i}'     for i in (4,5,6)]
            for c in rcot_cols:
                flat[f'rcot_current_{c}'] = float(row_current.get(c, np.nan))
            for k, v in res['rcot_opt'].items():
                flat[f'rcot_opt_{k}'] = float(v)
            for p in ['Ethylene','Propylene','MixedC4','RPG','Hydrogen','Tail_Gas']:
                flat[f'{p}_current_tph'] = float(y0.get(f'{p}_prod_t+1', np.nan))
                flat[f'{p}_opt_tph']     = float(y_opt.get(f'{p}_prod_t+1', np.nan))
                flat[f'{p}_delta_tph']   = flat[f'{p}_opt_tph'] - flat[f'{p}_current_tph']

            (OUT_DIR / "multi_snapshots").mkdir(exist_ok=True, parents=True)
            snap = dict(
                timestamp=str(ts), status=res.get('status'),
                margins=dict(current=m_base, optimal=m_opt, delta=d_m),
                rcot_opt=res.get('rcot_opt', {}),
                yields_current={k: float(y0.get(k, np.nan)) for k in y0.keys()},
                yields_opt={k: float(y_opt.get(k, np.nan)) for k in y_opt.keys()},
            )
            (OUT_DIR / "multi_snapshots" / f"multi_{ts:%Y%m%d_%H%M}.json").write_text(json.dumps(snap, indent=2))

            # ---- Attribute (anchored deltas) ----
            def _flow_for_knob(knob: str, r: pd.Series) -> float:
                if knob.startswith('RCOT_chamber'):
                    ch = int(knob.replace('RCOT_chamber',''))
                    return float(r.get(f'Naphtha_chamber{ch}', 0.0))
                if knob.startswith('RCOT_naphtha_chamber'):
                    ch = int(knob.replace('RCOT_naphtha_chamber',''))
                    return float(r.get(f'Naphtha_chamber{ch}', 0.0))
                if knob.startswith('RCOT_gas_chamber'):
                    ch = int(knob.replace('RCOT_gas_chamber',''))
                    return float(r.get(f'Gas Feed_chamber{ch}', 0.0))
                return 0.0

            rcot_knobs = rcot_cols
            moves = []
            for knob in rcot_knobs:
                rc_curr = float(row_current.get(knob, np.nan))
                rc_opt  = float(res['rcot_opt'].get(knob, rc_curr))
                flow    = _flow_for_knob(knob, row_current)
                active_flag = bool((flow > 1.0) and np.isfinite(rc_curr) and (rc_curr >= 800.0))
                moves.append(dict(timestamp=ts, knob=knob, flow_tph=flow,
                                  rcot_current_C=rc_curr, rcot_opt_C=rc_opt,
                                  delta_C=(rc_opt - rc_curr), active=active_flag))
            df_moves = pd.DataFrame(moves)
            (OUT_DIR / "rcot_moves").mkdir(exist_ok=True, parents=True)
            if not df_moves.empty:
                df_moves.to_csv(OUT_DIR / "rcot_moves" / f"rcot_moves_{ts:%Y%m%d_%H%M}.csv", index=False)
                MOVES_CSV = OUT_DIR / "rcot_moves_all.csv"
                df_moves.to_csv(MOVES_CSV, mode='a', header=not Path(MOVES_CSV).exists(), index=False)

            # ---- Prices ----
            def _prices_at(ts: pd.Timestamp, pp: opt.PriceProvider):
                pr_c = {p: float(pp.get(ts, {'Ethylene':'Ethylene','Propylene':'Propylene','MixedC4':'Mixed C4','RPG':'RPG','Hydrogen':'Hydrogen','Tail_Gas':'Tail Gas'}[p], 0.0))
                        for p in ['Ethylene','Propylene','MixedC4','RPG','Hydrogen','Tail_Gas']}
                pr_f = {
                    'PN':        float(pp.get(ts, 'PN', 0.0)),
                    'Gas Feed':  float(pp.get(ts, 'Gas Feed', 0.0)),
                    'LPG':       float(pp.get(ts, 'LPG', float(pp.get(ts,'Gas Feed',0.0)))),
                    'MX Offgas': float(pp.get(ts, 'MX Offgas', 0.0)),
                    'Fuel Gas':  float(pp.get(ts, 'Fuel Gas', 0.0)),
                    'Tail Gas':  float(pp.get(ts, 'Tail Gas', 0.0)),
                }
                return pr_c, pr_f

            pr_c, pr_f = _prices_at(ts, pp)

            def _revenue(yields_abs, pr_c):
                return sum(float(yields_abs.get(f'{p}_prod_t+1',0.0))*pr_c[p] for p in ['Ethylene','Propylene','MixedC4','RPG','Hydrogen','Tail_Gas'])

            def _feeds(r, pr_f):
                naph = sum(float(r.get(f'Naphtha_chamber{i}',0.0)) for i in range(1,7))
                gasC = sum(float(r.get(f'Gas Feed_chamber{i}',0.0)) for i in (4,5,6))
                fresh_lpg = float(r.get('FreshFeed_C3 LPG',0.0))
                fresh_off = float(r.get('FreshFeed_MX Offgas',0.0))
                if fresh_lpg>0 or fresh_off>0:
                    return naph*pr_f['PN'] + fresh_lpg*pr_f['LPG'] + fresh_off*pr_f['MX Offgas']
                return naph*pr_f['PN'] + gasC*pr_f['Gas Feed']

            def _recycle(yields_abs, pr_f):
                eth = float(yields_abs.get('Ethane_prod_t+1',0.0)); pro = float(yields_abs.get('Propane_prod_t+1',0.0))
                return (eth + pro) * pr_f['LPG']

            # ΔFG energy fixed baseline
            def _per_side_rc_eff(r: pd.Series):
                def _mean(keys, fkeys, rc_min=800.0, flow_thr=1.0):
                    pairs = []
                    for k, fk in zip(keys, fkeys):
                        rc = float(r.get(k, np.nan)); f = float(r.get(fk, 0.0))
                        if np.isfinite(rc) and rc >= rc_min and f > flow_thr:
                            pairs.append((rc, f))
                    if not pairs: return np.nan
                    w = sum(f for _, f in pairs)
                    return sum(rc*f for rc,f in pairs)/w if w>0 else np.nan
                rc13   = _mean([f'RCOT_chamber{i}'         for i in (1,2,3)],
                               [f'Naphtha_chamber{i}'      for i in (1,2,3)])
                rcn456 = _mean([f'RCOT_naphtha_chamber{i}' for i in (4,5,6)],
                               [f'Naphtha_chamber{i}'      for i in (4,5,6)])
                rcg456 = _mean([f'RCOT_gas_chamber{i}'     for i in (4,5,6)],
                               [f'Gas Feed_chamber{i}'     for i in (4,5,6)])
                rc_n_eff = np.nanmean([rc13, rcn456]) if (np.isfinite(rc13) or np.isfinite(rcn456)) else np.nan
                rc_g_eff = rcg456
                naph = sum(float(r.get(f'Naphtha_chamber{i}', 0.0))  for i in range(1,7))
                gas  = sum(float(r.get(f'Gas Feed_chamber{i}', 0.0)) for i in (4,5,6))
                D    = max(float(naph + gas), 1e-9)
                wN, wG = (naph / D), (gas / D)
                return rc_n_eff, rc_g_eff, wN, wG, D

            def _fg_abs(r: pd.Series):
                for sk in ('Fuel_Gas','Fuel Gas','FG','FuelGas','Tail Gas','Tail_Gas'):
                    try: return float(_spyro_ts(r, sk, ctx=None))
                    except Exception: pass
                return 0.0

            def _override_rcot(r: pd.Series, rc_n=None, rc_g=None) -> pd.Series:
                rr = r.copy()
                if rc_n is not None:
                    for ch in (1,2,3): rr[f'RCOT_chamber{ch}'] = float(rc_n)
                    for ch in (4,5,6): rr[f'RCOT_naphtha_chamber{ch}'] = float(rc_n)
                if rc_g is not None:
                    for ch in (4,5,6): rr[f'RCOT_gas_chamber{ch}'] = float(rc_g)
                return rr

            rc_n0, rc_g0, _, _, _ = _per_side_rc_eff(row_current)

            def _dfg_energy(row_like, yields_abs, rc_n0, rc_g0):
                rc_n, rc_g, wN, wG, D = _per_side_rc_eff(row_like)
                if not np.isfinite(rc_n): rc_n = float(fg_consts.rc_ref_naph)
                if not np.isfinite(rc_g): rc_g = float(fg_consts.rc_ref_gas)
                E_abs = float(yields_abs.get('Ethylene_prod_t+1',0.0))
                P_abs = float(yields_abs.get('Propylene_prod_t+1',0.0))
                FG_ab = _fg_abs(row_like)
                rE, rP, rFG = E_abs/D, P_abs/D, FG_ab/D
                r_base = _override_rcot(row_like, rc_n=rc_n0, rc_g=rc_g0)
                base_E = float(_spyro_ts(r_base, 'Ethylene', None))/D
                base_P = float(_spyro_ts(r_base, 'Propylene', None))/D
                base_FG = _fg_abs(r_base)/D
                Cpw = float(fg_consts.cp_wavg_kcal_per_ton_K); HHV = float(fg_consts.fuel_gas_heat_content_kcal_per_ton)
                rc_term = Cpw * (wN*(rc_n-rc_n0) + wG*(rc_g-rc_g0))
                S = rc_term + (rE-base_E)*float(fg_consts.dH_eth_kcal_per_ton) \
                            + (rP-base_P)*float(fg_consts.dH_prop_kcal_per_ton) \
                            + (rFG-base_FG)*float(fg_consts.dH_fg_kcal_per_ton)
                return float((S/HHV)*D)

            # ---- Summary/Audit ----
            items = pd.DataFrame([
                dict(product=p.replace('MixedC4','Mixed C4').replace('Tail_Gas','Tail Gas'),
                     qty_before=float(y0.get(f'{p}_prod_t+1',0.0)),
                     qty_after= float(y_opt.get(f'{p}_prod_t+1',0.0)),
                     price=pr_c[p],
                     rev_before=float(y0.get(f'{p}_prod_t+1',0.0))*pr_c[p],
                     rev_after=float(y_opt.get(f'{p}_prod_t+1',0.0))*pr_c[p],
                     rev_delta=(float(y_opt.get(f'{p}_prod_t+1',0.0))-float(y0.get(f'{p}_prod_t+1',0.0)))*pr_c[p]
                     )
                for p in ['Ethylene','Propylene','MixedC4','RPG','Hydrogen','Tail_Gas']
            ])

            rev_b = _revenue(y0, pr_c);     rev_a = _revenue(y_opt, pr_c)
            feed_b= _feeds(row_current, pr_f);     feed_a= _feeds(row_opt, pr_f)
            rec_b = _recycle(y0, pr_f);     rec_a=  _recycle(y_opt, pr_f)
            dfg_b = _dfg_energy(row_current, y0, rc_n0, rc_g0);   dfg_a = _dfg_energy(row_opt, y_opt, rc_n0, rc_g0)
            fg_price = pr_f.get('Fuel Gas', pr_f.get('Tail Gas', 0.0))
            fgc_b = dfg_b * fg_price;       fgc_a = dfg_a * fg_price
            m_b   = rev_b - feed_b + rec_b - fgc_b
            m_a   = rev_a - feed_a + rec_a - fgc_a

            print("\n=== PRICE SNAPSHOT @", ts, "===")
            print({
                'Ethylene':  pr_c['Ethylene'],
                'Propylene': pr_c['Propylene'],
                'Mixed C4':  pr_c['MixedC4'],
                'RPG':       pr_c['RPG'],
                'Hydrogen':  pr_c['Hydrogen'],
                'Tail Gas':  pr_f['Tail Gas'],
                'Fuel Gas':  pr_f['Fuel Gas'],
                'PN':        pr_f['PN'],
                'Gas Feed':  pr_f['Gas Feed'],
                'LPG':       pr_f['LPG'],
                'MX Offgas': pr_f['MX Offgas'],
            })

            print("\n=== BEFORE vs AFTER (per product) ===")
            print(items[['product','qty_before','qty_after','price','rev_before','rev_after','rev_delta']]
                  .sort_values('product').to_string(index=False, float_format=lambda x: f"{x:,.3f}"))

            print("\n=== SUMMARY ($/h) ===")
            print(f"Revenue      : {rev_b:,.2f}  →  {rev_a:,.2f}   (Δ {rev_a-rev_b:+,.2f})")
            print(f"Feed cost    : {feed_b:,.2f} →  {feed_a:,.2f}  (Δ {feed_a-feed_b:+,.2f})")
            print(f"Recycle cred.: {rec_b:,.2f} →  {rec_a:,.2f}  (Δ {rec_a-rec_b:+,.2f})")
            print(f"ΔFG (tph)    : {dfg_b:.6f} → {dfg_a:.6f} (Δ {dfg_a-dfg_b:+.6f}) × Fuel Gas={fg_price:,.2f} → FG cost Δ: {(fgc_a-fgc_b):+,.2f}")
            print(f"MARGIN       : {m_b:,.2f} → {m_a:,.2f} (Δ {m_a-m_b:+,.2f})")

            (OUT_DIR / "audits").mkdir(exist_ok=True, parents=True)
            items.to_csv(OUT_DIR / "audits" / f"audit_items_{ts:%Y%m%d_%H%M}.csv", index=False)
            summary = dict(
                ts=str(ts),
                prices=dict(canonical=pr_c, feeds=pr_f),
                revenue_before=rev_b, revenue_after=rev_a,
                feed_before=feed_b, feed_after=feed_a,
                recycle_before=rec_b, recycle_after=rec_a,
                dfg_tph_before=dfg_b, dfg_tph_after=dfg_a, fuelgas_price=fg_price,
                fg_cost_before=fgc_b, fg_cost_after=fgc_a,
                margin_before=m_b, margin_after=m_a,
                rcot_opt=res.get('rcot_opt',{})
            )
            (OUT_DIR / "audits" / f"audit_summary_{ts:%Y%m%d_%H%M}.json").write_text(json.dumps(summary, indent=2))

            # Save curves (overrides-only α; rc0-injected)
            try:
                setter = {'LF_NAPH': gpmod.rcot_setter_lf_naph,
                          'GF_GAS':  gpmod.rcot_setter_gf_gas,
                          'GF_HYB_NAPH': gpmod.rcot_setter_hybrid}[geom]
                rc_lo, rc_hi = RC_BOUNDS.get(geom, (810, 853))
                rc_grid = _rc_grid_for(row_current, geom, rc_lo, rc_hi, points=15)
                curve, x_row = gpmod.anchored_curve_at_ts(
                    gp=gp, X_12h=X_12h, merged_lims=merged_lims, pipeline=pipeline,
                    ml=ml_cached, ts=ts, rcot_setter=setter, rc_grid=rc_grid,
                    use_gp_delta=True, alpha=0.0
                )
                (OUT_DIR / "curves").mkdir(exist_ok=True, parents=True)
                curve.to_csv(OUT_DIR / "curves" / f"curve_{ts:%Y%m%d_%H%M}_{geom}.csv", index=False)
                # slope fidelity at save time
                rows = []
                for p in gpmod.PRODUCTS:
                    sc, cov = _robust_slope_metrics(curve, p)
                    a_col, c_col = f'{p}_ANCHOR_tph', f'{p}_CORR_tph'
                    diff = (curve[c_col].astype(float) - curve[a_col].astype(float)).abs().to_numpy(float) if (a_col in curve and c_col in curve) else np.array([np.nan])
                    rows.append({'timestamp': ts, 'geometry': geom, 'product': p,
                                 'slope_corr': sc, 'sign_cov': cov, 'anchor_miss_min': float(np.nanmin(diff))})
                pd.DataFrame(rows).to_csv(OUT_DIR / "curves" / f"audit_{ts:%Y%m%d_%H%M}_{geom}.csv", index=False)
            except Exception as e:
                print(f"[WARN] Curve/audit save failed at {ts}: {e}")

            # persist recommendations row
            rec_rows.append(flat)

            # ---------- CLOSED-LOOP: apply setpoints & update simulated lags ----------
            if mode == 'closed_loop':
                knobs_applied = {k: float(v) for k, v in res['rcot_opt'].items()}

                if apply_timing == 'next_stamp':
                    idx = X_state.index
                    next_mask = idx > ts
                    if next_mask.any():
                        t1 = idx[next_mask][0]
                        for knob, val in knobs_applied.items():
                            if knob in X_state.columns:
                                X_state.loc[idx >= t1, knob] = val
                        print(f"[APPLIED next_stamp] {ts} -> {t1}")
                        klist = list(knobs_applied.keys())
                        print("  rcots@ts   :", X_state.loc[ts,  klist].to_dict())
                        print("  rcots@next :", X_state.loc[t1, klist].to_dict())
                else:
                    next_day = ts.normalize() + pd.Timedelta(days=1)
                    for knob, val in knobs_applied.items():
                        if knob in X_state.columns:
                            X_state.loc[X_state.index >= next_day, knob] = val
                    print(f"[SCHEDULED next_day] {ts} -> {next_day}: {knobs_applied}")

                # write simulated corrected yields for this stamp to Y_sim_state (for ML lags)
                if Y_sim_state is not None:
                    for c in TARGET_COLS:
                        if c in Y_sim_state.columns:
                            Y_sim_state.at[ts, c] = float(res['yields_opt'].get(c, np.nan))

        # append day’s rows
        if rec_rows:
            df_new = pd.DataFrame(rec_rows)
            _safe_merge_csv(OUT_DIR / f"rcot_recommendations{cache_tag}.csv", df_new, key='timestamp')

    # --------- END OF RUN: counterfactual check ---------
    rec_path = OUT_DIR / f"rcot_recommendations{cache_tag}.csv"
    if rec_path.exists():
        recs = pd.read_csv(rec_path, parse_dates=['timestamp']).sort_values('timestamp')
        schedule = build_rcot_schedule_from_recs(recs)
        X_sim = apply_schedule_to_X(X_12h, schedule, start=start, end=end, hold="hold_until_next")
        Y_sim, M_sim = simulate_path_corrected(
            X_sim=X_sim, merged_lims=merged_lims, pipeline=pipeline,
            gp=gp, gps_dict=gps_dict, feature_cols_gp=feature_cols_gp,
            price_provider=pp, total_spyro_yield_for_now=total_spyro_yield_for_now,
            fg_consts=fg_consts, alpha_overrides=None,
            start=start, end=end
        )
        (OUT_DIR / "counterfactual").mkdir(exist_ok=True, parents=True)
        X_sim.to_csv(OUT_DIR / "counterfactual" / "X_sim_rcot_applied.csv")
        Y_sim.to_csv(OUT_DIR / "counterfactual" / "Y_sim_corrected.csv")
        M_sim.to_csv(OUT_DIR / "counterfactual" / "M_sim_margin.csv")

        if 'margin_baseline_per_h' in recs.columns:
            compare = (pd.DataFrame({'margin_hist': recs.set_index('timestamp')['margin_baseline_per_h']})
                        .join(M_sim.rename(columns={'margin_per_h':'margin_sim'}), how='inner'))
            print("\n--- Margin hist vs sim (first 6 rows) ---")
            print(compare.head(6).to_string())

    print("✅ Production run complete.")


## 1. Data Loading

In [2]:
import src.data_loading as dl; reload(dl)
from src.data_loading import DataPaths, ResampleConfig, DataPipeline

paths = DataPaths(
    input_dir=Path("input"), inter_dir=Path("intermediate"),
    prod_excel="1. 생산량 Data_'23.07~'25.05_R1_송부용.xlsx",
    furn_excel="2. Furnace Data_'23.07~'25.05_R0.xlsx",
    nap_excel ="Nap Feed 조성분석값.xlsx",
    gas_excel ="Gas Feed 조성분석값.xlsx",
    recycle_excel="6. 에탄 및 프로판 데이터.xlsx",
    # cost_excel="마진가격_vModel_v250625.xlsx",
    price_csv= "price.csv",
    util_excel="#1ECU 유틸리티사용량일별데이터.xlsx",
    fresh_excel="7. Gas Furnace Feed Data_'23.07~'25.05_r2.xlsx",

    # PKL caches (optional)
    prod_pkl="df_production_v4.pkl", furn_pkl="furnace.pkl",
    nap_pkl ="df_feed_naptha.pkl", gas_pkl ="df_feed_gas.pkl",
    fresh_pkl= 'df_feed_fresh_v3.pkl', rec_pkl ="df_recycle.pkl",
    prod_header=2, furn_header=2, nap_header=1, gas_header=1, rec_header=4, fresh_header=3
)
cfg = ResampleConfig(hour_freq='h', win12_freq='12h', win12_offset='9h')


feature_rename = {
    # map your util feature names → canonical (only if you use util models)
    'Naph': 'Naphtha_chamber1', 'T-DAO': 'T-DAO_chamber1', 'DS': 'DS_chamber1',
    'RCOT Ave.': 'RCOT_chamber1', 'Excess O2': "Excess O2_chamber1",
    'Naph.1': 'Naphtha_chamber2', 'T-DAO.1': 'T-DAO_chamber2','DS.1': 'DS_chamber2',
    'RCOT Ave..1': 'RCOT_chamber2', 'Excess O2.1': "Excess O2_chamber2",
    'Naph.2': 'Naphtha_chamber3', 'T-DAO.2': 'T-DAO_chamber3','DS.2': 'DS_chamber3',
    'RCOT Ave..2': 'RCOT_chamber3', 'Excess O2.2': "Excess O2_chamber3",
    'Naph.3': 'Naphtha_chamber4', 'GAS': 'Gas Feed_chamber4','DS.3': 'DS_chamber4',
    'RCOT Ave..3': 'RCOT_chamber4', 'Excess O2.3': "Excess O2_chamber4",
    'Naph.4': 'Naphtha_chamber5', 'GAS.1': 'Gas Feed_chamber5','DS.4': 'DS_chamber5',
    'RCOT Ave..4': 'RCOT_chamber5', 'Excess O2.4': "Excess O2_chamber5",
    'Naph.5': 'Naphtha_chamber6', 'GAS.2': 'Gas Feed_chamber6','DS.5': 'DS_chamber6',
    'RCOT Ave..5': 'RCOT_chamber6', 'Excess O2.5': "Excess O2_chamber6",
}
target_rename  = { 'Unnamed: 36':'steam','ECU F/G':'fuel_gas','ECU Elec..1':'electricity' }

dp = DataPipeline(paths, cfg).run(feature_rename, target_rename)
art = dp.artifacts()
X_12h, Y_12h, util_df, prices_df = art['X_12h'], art['Y_12h'], art['util_df'], art['price_df']

# clamp horizon (LIMS)
X_12h = X_12h.loc[:END]
Y_12h = Y_12h.loc[:END]
print(X_12h.shape, Y_12h.shape)


(1376, 100) (1376, 9)


c:\Users\user\Downloads\EFOM\src\data_loading.py:425: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['month'] = df['timestamp'].dt.to_period('M').dt.to_timestamp()


## 2) LIMS + SRTO pipeline

In [3]:
from src.data_loading import load_feed_data

merged_lims = load_feed_data(
    nap_path=paths.input_dir / "복사본 (2024-25) ECU 투입 납사 세부성상-wt%.xlsx",
    gas_path=paths.input_dir / "Gas Feed 조성분석값.xlsx", header=1
)
merged_lims['date'] = pd.to_datetime(merged_lims['date'], errors='coerce')
merged_lims = merged_lims.dropna(subset=['date']).sort_values('date')

gas_cols = [c for c in ['Ethylene','Ethane','Propylene','Propane','n-Butane','i-Butane'] if c in merged_lims.columns]
zr = (merged_lims[gas_cols].sum(axis=1) == 0)
merged_lims.loc[zr, gas_cols] = np.nan
merged_lims[gas_cols] = merged_lims[gas_cols].ffill().bfill()
merged_lims = merged_lims.iloc[4:]  # keep (as you had)

# SRTO DLL pipeline (plant spot)
from src.srto_pipeline import SRTOConfig, RCOTSweepConfig, FeedConfig, SRTOPipeline
from src.srto_components import component_index, MW
dll_folder = Path(r"C:\Program Files\Pyrotec\SRTO")
selected_spy7 = [
    dll_folder / "01. GF_HYBRID MODE_SRTO7_NAPH.SPY7",
    dll_folder / "04. LF_NAPH MODE_SRTO7.SPY7",
    dll_folder / "07. GF_GAS MODE_SRTO7.SPY7",
]
srto_config  = SRTOConfig(dll_folder, selected_spy7, component_index, MW)
sweep_config = RCOTSweepConfig(rcot_min=790.0, rcot_max=900.0, rcot_step=2.0,
                               chunk_size=10, n_jobs=6, save_checkpoints=True)
feed_config  = FeedConfig(gas_components=gas_cols)
pipeline     = SRTOPipeline(srto_config, sweep_config, feed_config)


c:\Users\user\Downloads\EFOM\src\data_loading.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feed_gas['date'] = pd.to_datetime(feed_gas['date'], errors='coerce')


In [4]:
pona_cols = ['Paraffins','Olefins','Naphthenes','Aromatics']

# Merge X_12h with merged_lims per AM/PM rule:
# hour < 12 -> use previous day's LIMS, else use same day's LIMS; fallback to asof if day-join misses rows.
x = X_12h.copy()
ts = x.index.to_series()
lims_date = ts.dt.normalize()
lims_date = lims_date.where(ts.dt.hour >= 12, lims_date - pd.Timedelta(days=1))
x = x.assign(lims_date=lims_date.values)

m = merged_lims.copy()
m['lims_date'] = pd.to_datetime(m['date'], errors='coerce').dt.normalize()
m_daily = m.sort_values('date').groupby('lims_date', as_index=False).last()

keep = pona_cols + gas_cols
m_sel = m_daily[['lims_date'] + keep]

xr = x.reset_index()
idx_name = xr.columns[0]   # original timestamp column name
merged = xr.merge(m_sel, on='lims_date', how='left').set_index(idx_name)
merged.index.name = X_12h.index.name

# fallback: fill any remaining NaNs with the most recent earlier merged_lims record
if merged[keep].isna().any().any():
    mr = m.sort_values('date')[['date'] + keep].rename(columns={'date': 'lims_ts'})
    xr_ts = xr.copy(); xr_ts['ts'] = pd.to_datetime(xr_ts[idx_name])
    asof = pd.merge_asof(xr_ts.sort_values('ts'),
                         mr.sort_values('lims_ts'),
                         left_on='ts', right_on='lims_ts', direction='backward').set_index(idx_name)
    for c in keep:
        merged[c] = merged[c].fillna(asof[c])

X_12h_with_lims = X_12h.join(merged[keep])
print('X_12h_with_lims shape:', X_12h_with_lims.shape)
print('Missing after merge:', X_12h_with_lims[keep].isna().sum().to_dict())

gas_rename = {c: f"{c}_gas" for c in gas_cols}  # e.g., 'Ethane' -> 'Ethane_gas'
X_12h_with_lims = X_12h_with_lims.rename(columns=gas_rename)


X_12h_with_lims shape: (1376, 110)
Missing after merge: {'Paraffins': 368, 'Olefins': 368, 'Naphthenes': 368, 'Aromatics': 368, 'Ethylene': 368, 'Ethane': 368, 'Propylene': 368, 'Propane': 368, 'n-Butane': 368, 'i-Butane': 368}


C:\Users\user\AppData\Local\Temp\ipykernel_22912\562457764.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged[c] = merged[c].fillna(asof[c])


## 3) Rolling 


In [5]:
# --- lightweight memo around total_spyro_yield_for_now ---
class SpyroMemo:
    def __init__(self, fn, key_cols=None, decimals=4, maxsize=200000):
        self.fn = fn
        self.key_cols = tuple(key_cols) if key_cols is not None else None  # ensure tuple
        self.dec = decimals
        self.cache = {}
        self.maxsize = maxsize

    def _select_cols(self, row: pd.Series):
        if self.key_cols is None:
            # stable, hashable selection
            return tuple(
                c for c in row.index
                if c.startswith('RCOT')
                or c.startswith('Naphtha_chamber')
                or c.startswith('Gas Feed_chamber')
            )
        return self.key_cols

    def _to_num(self, x):
        try:
            v = float(x)
        except Exception:
            v = 0.0
        # handle NaN
        if v != v:  # NaN check without importing math
            v = 0.0
        return round(v, self.dec)

    def _sig(self, row: pd.Series, short_key: str):
        cols = self._select_cols(row)                # tuple, hashable
        vals = tuple(self._to_num(row.get(c, 0.0)) for c in cols)
        return (short_key, cols, vals)               # fully hashable

    def __call__(self, row: pd.Series, short_key: str, ctx=None):
        k = self._sig(row, short_key)
        v = self.cache.get(k)
        if v is not None:
            return v
        v = self.fn(row, short_key, ctx=ctx)
        if len(self.cache) < self.maxsize:
            self.cache[k] = v
        return v

# wrap it
# total_spyro_yield_for_now = SpyroMemo(total_spyro_yield_for_now)


In [6]:
# RCOT groups so setters touch the right columns
gpmod.set_rcot_groups_from_columns(X_12h.columns)

# Prices
pp = opt.PriceProvider(prices_df)

# If you haven't defined this yet, define a minimal SPYRO wrapper (ctx not required)
_SHORT_TO_SRTO = {
    'Ethylene':'Ethylene','Propylene':'Propylene','MixedC4':'MixedC4','RPG':'RPG',
    'Ethane':'Ethane','Propane':'Propane',
    'Fuel_Gas':'Fuel_Gas','Fuel Gas':'Fuel_Gas','FG':'Fuel_Gas','FuelGas':'Fuel_Gas',
    'Tail Gas':'Tail_Gas', 'Tail_Gas' :'Tail_Gas'
}
def total_spyro_yield_for_now(row_like: pd.Series, short_key: str, ctx=None) -> float:
    ts = getattr(row_like, 'name', None)
    if ts is None: return 0.0
    comp_row = merged_lims.loc[merged_lims['date'] <= ts].iloc[-1]
    spot = pipeline.predict_spot_plant(row_like, comp_row, feed_thr=0.1)
    if spot.get('status') != 'ok': return 0.0
    key = _SHORT_TO_SRTO.get(short_key, short_key)
    return float(spot['totals_tph'].get(key, 0.0))

# Excel-delta margin wrapper (uses opt.delta_fg_excel by default)
margin_excel = opt.make_margin_fn_excel_delta(
    price_provider=pp,
    total_spyro_yield_for_now=total_spyro_yield_for_now,
    spyro_ctx=None,                                  # SPYRO_CTX can be None
    fg_constants=opt.FuelGasConstants())

# Bounds by geometry
RC_BOUNDS = {
    'LF_NAPH': (830.0, 853.0),
    'GF_GAS':  (860.0, 890.0),
    'GF_HYB_NAPH': (830.0, 853.0),
}


# wrap it
memo_spyro = SpyroMemo(total_spyro_yield_for_now)


In [7]:
first_eval_day = pd.Timestamp('2025-01-01')
train_end = (X_12h.index[(X_12h.index >= first_eval_day) &
                         (X_12h.index < first_eval_day + pd.Timedelta(days=1))][-1]
             - pd.Timedelta(hours=12))
train_start = train_end - LOOKBACK_6M
print("ML/GP window:", train_start, "→", train_end)

window_rows = ((X_12h.index >= train_start) & (X_12h.index < train_end)).sum()
print("Rows in lookback window:", window_rows, "(MIN_TR_ROWS =", MIN_TR_ROWS, ")")


ML/GP window: 2024-07-05 09:00:00 → 2025-01-01 09:00:00
Rows in lookback window: 360 (MIN_TR_ROWS = 180 )


In [ ]:
TARGET_COLS_ML = TARGET_COLS

# Example daily run for the whole horizon (or pass a shorter end date)
run_production(
    X_12h=X_12h, Y_12h=Y_12h,
    merged_lims=merged_lims, pipeline=pipeline,
    prices_df=prices_df,
    total_spyro_yield_for_now=memo_spyro,
    start=pd.Timestamp('2025-01-01'), end=pd.Timestamp('2025-05-19'),
    mode='closed_loop',
    closed_loop_opts=dict(
        apply_timing='next_stamp',          # or 'next_stamp'
        hold_policy='hold_until_next',    # step & hold
        ml_train_mode='simulated',       # train ML on historical windows (fast/stable)
        gp_train_mode='simulated',       # train GP on historical windows
        cache_tag='_sim'                  # keep caches separate
    )
)                                                    


In [ ]:
output sample

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 572.99
RCOT* (per chamber):
  RCOT_chamber2              835.01 →  840.01  (Δ +5.00)
  RCOT_chamber3              835.00 →  840.00  (Δ +5.00)
  RCOT_gas_chamber5          880.02 →  885.02  (Δ +5.00)
  RCOT_naphtha_chamber6      839.99 →  844.99  (Δ +5.00)

=== PRICE SNAPSHOT @ 2025-01-01 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      64.632     66.182   826.920  53,445.614 54,727.089  1,281.474
 Hydrogen       1.680      1.769 2,030.931   3,411.929  3,592.006    180.077
 Mixed C4      23.833     23.931   879.236  20,955.185 21,040.694     85.509
Propylene      35.821     35.515   805.800  28,864.537 28,618.242   -246.295
      RPG      53.810     52.619   687.199  36,978.406 36,159.761   -818.645
 Tail Gas      31.522     32.369   730.140  23,015.330 23,634.150    618.819

=== SUMMARY ($/h) ===
Revenue      : 166,671.00  →  167,771.94   (Δ +1,100.94)
Feed cost    : 130,560.02 →  130,560.02  (Δ +0.00)
Recycle cred.: 13,876.62 →  13,426.98  (Δ -449.64)
ΔFG (tph)    : -0.241236 → -0.115324 (Δ +0.125913) × Fuel Gas=621.92 → FG cost Δ: +78.31
MARGIN       : 50,137.63 → 50,710.63 (Δ +572.99)
[APPLIED next_stamp] 2025-01-01 09:00:00 -> 2025-01-01 21:00:00
  rcots@ts   : {'RCOT_chamber2': 835.0071319145932, 'RCOT_chamber3': 835.0042896947247, 'RCOT_gas_chamber5': 880.0180056200428, 'RCOT_naphtha_chamber6': 839.9901574771825}
  rcots@next : {'RCOT_chamber2': 840.0071319074935, 'RCOT_chamber3': 840.0042896876739, 'RCOT_gas_chamber5': 885.0180056112196, 'RCOT_naphtha_chamber6': 844.990157463037}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 489.01
RCOT* (per chamber):
  RCOT_chamber2              840.01 →  845.01  (Δ +5.00)
  RCOT_chamber3              840.00 →  845.00  (Δ +5.00)
  RCOT_gas_chamber5          885.02 →  890.00  (Δ +4.98)
  RCOT_naphtha_chamber6      844.99 →  849.99  (Δ +5.00)

=== PRICE SNAPSHOT @ 2025-01-01 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      64.802     66.237   826.920  53,586.023 54,772.572  1,186.549
 Hydrogen       1.667      1.754 2,030.931   3,385.317  3,561.760    176.443
 Mixed C4      23.788     23.854   879.236  20,915.540 20,972.967     57.426
Propylene      35.567     35.190   805.800  28,660.118 28,356.283   -303.835
      RPG      53.969     52.902   687.199  37,087.536 36,354.176   -733.360
 Tail Gas      31.460     32.287   730.140  22,970.500 23,574.374    603.873

=== SUMMARY ($/h) ===
Revenue      : 166,605.03  →  167,592.13   (Δ +987.10)
Feed cost    : 130,899.51 →  130,899.51  (Δ +0.00)
Recycle cred.: 13,993.78 →  13,567.09  (Δ -426.70)
ΔFG (tph)    : -0.372161 → -0.257362 (Δ +0.114799) × Fuel Gas=621.92 → FG cost Δ: +71.40
MARGIN       : 49,930.76 → 50,419.77 (Δ +489.01)
[APPLIED next_stamp] 2025-01-01 21:00:00 -> 2025-01-02 09:00:00
  rcots@ts   : {'RCOT_chamber2': 840.0071319074935, 'RCOT_chamber3': 840.0042896876739, 'RCOT_gas_chamber5': 885.0180056112196, 'RCOT_naphtha_chamber6': 844.990157463037}
  rcots@next : {'RCOT_chamber2': 845.0071318362533, 'RCOT_chamber3': 845.0042896167266, 'RCOT_gas_chamber5': 889.9999999112305, 'RCOT_naphtha_chamber6': 849.990157463037}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 251.46
RCOT* (per chamber):
  RCOT_chamber2              845.01 →  850.01  (Δ +5.00)
  RCOT_chamber3              845.00 →  850.00  (Δ +5.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber6      849.99 →  853.00  (Δ +3.01)

=== PRICE SNAPSHOT @ 2025-01-02 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      64.777     65.693   826.920  53,565.467 54,322.604    757.137
 Hydrogen       1.678      1.714 2,030.931   3,407.349  3,481.707     74.358
 Mixed C4      23.840     23.757   879.236  20,960.724 20,888.367    -72.357
Propylene      35.535     35.270   805.800  28,634.145 28,420.530   -213.615
      RPG      54.228     53.325   687.199  37,265.225 36,644.970   -620.255
 Tail Gas      31.523     32.025   730.140  23,016.278 23,382.498    366.220

=== SUMMARY ($/h) ===
Revenue      : 166,849.19  →  167,140.68   (Δ +291.49)
Feed cost    : 130,926.30 →  130,926.30  (Δ +0.00)
Recycle cred.: 13,954.26 →  13,958.58  (Δ +4.32)
ΔFG (tph)    : -0.370704 → -0.299392 (Δ +0.071313) × Fuel Gas=621.92 → FG cost Δ: +44.35
MARGIN       : 50,107.70 → 50,359.15 (Δ +251.46)
[APPLIED next_stamp] 2025-01-02 09:00:00 -> 2025-01-02 21:00:00
  rcots@ts   : {'RCOT_chamber2': 845.0071318362533, 'RCOT_chamber3': 845.0042896167266, 'RCOT_gas_chamber5': 889.9999999112305, 'RCOT_naphtha_chamber6': 849.990157463037}
  rcots@next : {'RCOT_chamber2': 850.007131765306, 'RCOT_chamber3': 850.0042895457306, 'RCOT_gas_chamber5': 889.9999999118164, 'RCOT_naphtha_chamber6': 852.9999998581055}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 114.54
RCOT* (per chamber):
  RCOT_chamber2              850.01 →  853.00  (Δ +2.99)
  RCOT_chamber3              850.00 →  853.00  (Δ +3.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-02 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      66.492     66.930   826.920  54,983.760 55,345.328    361.567
 Hydrogen       1.669      1.687 2,030.931   3,389.594  3,425.510     35.916
 Mixed C4      24.088     24.044   879.236  21,179.465 21,140.652    -38.812
Propylene      36.215     36.083   805.800  29,181.907 29,075.708   -106.199
      RPG      52.545     52.117   687.199  36,108.616 35,814.572   -294.043
 Tail Gas      31.943     32.180   730.140  23,323.103 23,495.673    172.569

=== SUMMARY ($/h) ===
Revenue      : 168,166.44  →  168,297.44   (Δ +131.00)
Feed cost    : 132,358.90 →  132,358.90  (Δ +0.00)
Recycle cred.: 14,063.73 →  14,067.23  (Δ +3.50)
ΔFG (tph)    : -0.323747 → -0.291648 (Δ +0.032098) × Fuel Gas=621.92 → FG cost Δ: +19.96
MARGIN       : 50,072.62 → 50,187.16 (Δ +114.54)
[APPLIED next_stamp] 2025-01-02 21:00:00 -> 2025-01-03 09:00:00
  rcots@ts   : {'RCOT_chamber2': 850.007131765306, 'RCOT_chamber3': 850.0042895457306, 'RCOT_gas_chamber5': 889.9999999118164, 'RCOT_naphtha_chamber6': 852.9999998581055}
  rcots@next : {'RCOT_chamber2': 852.9999999999274, 'RCOT_chamber3': 852.9999999999272, 'RCOT_gas_chamber5': 889.999999999914, 'RCOT_naphtha_chamber6': 852.9999999998546}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-03 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      67.209     67.209   826.920  55,576.647 55,576.647      0.000
 Hydrogen       1.675      1.675 2,030.931   3,402.823  3,402.823      0.000
 Mixed C4      24.196     24.196   879.236  21,273.586 21,273.586      0.000
Propylene      35.600     35.600   805.800  28,686.668 28,686.668      0.000
      RPG      52.460     52.460   687.199  36,050.318 36,050.318      0.000
 Tail Gas      32.207     32.207   730.140  23,515.640 23,515.640      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,505.68  →  168,505.68   (Δ +0.00)
Feed cost    : 131,736.96 →  131,736.96  (Δ +0.00)
Recycle cred.: 14,003.26 →  14,003.26  (Δ +0.00)
ΔFG (tph)    : -0.302530 → -0.302530 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 50,960.13 → 50,960.13 (Δ +0.00)
[APPLIED next_stamp] 2025-01-03 09:00:00 -> 2025-01-03 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999999274, 'RCOT_chamber3': 852.9999999999272, 'RCOT_gas_chamber5': 889.999999999914, 'RCOT_naphtha_chamber6': 852.9999999998546}
  rcots@next : {'RCOT_chamber2': 852.9999999927393, 'RCOT_chamber3': 852.9999999927246, 'RCOT_gas_chamber5': 889.9999999914404, 'RCOT_naphtha_chamber6': 852.9999999854639}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-03 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      68.431     68.431   826.920  56,587.342 56,587.342     -0.000
 Hydrogen       1.680      1.680 2,030.931   3,412.526  3,412.526     -0.000
 Mixed C4      24.322     24.322   879.236  21,385.043 21,385.043      0.000
Propylene      35.878     35.878   805.800  28,910.531 28,910.531      0.000
      RPG      51.078     51.078   687.199  35,100.946 35,100.946     -0.000
 Tail Gas      32.145     32.145   730.140  23,470.320 23,470.320     -0.000

=== SUMMARY ($/h) ===
Revenue      : 168,866.71  →  168,866.71   (Δ +0.00)
Feed cost    : 131,670.62 →  131,670.62  (Δ +0.00)
Recycle cred.: 13,995.92 →  13,995.92  (Δ +0.00)
ΔFG (tph)    : -0.160517 → -0.160517 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 51,291.84 → 51,291.84 (Δ +0.00)
[APPLIED next_stamp] 2025-01-03 21:00:00 -> 2025-01-04 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927393, 'RCOT_chamber3': 852.9999999927246, 'RCOT_gas_chamber5': 889.9999999914404, 'RCOT_naphtha_chamber6': 852.9999999854639}
  rcots@next : {'RCOT_chamber2': 852.9999999275391, 'RCOT_chamber3': 852.9999999276855, 'RCOT_gas_chamber5': 889.9999999154297, 'RCOT_naphtha_chamber6': 852.9999998552246}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-04 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.111     69.111   826.920  57,148.989 57,148.989      0.000
 Hydrogen       1.648      1.648 2,030.931   3,346.136  3,346.136      0.000
 Mixed C4      24.447     24.447   879.236  21,494.436 21,494.436      0.000
Propylene      35.658     35.658   805.800  28,732.853 28,732.853      0.000
      RPG      48.786     48.786   687.199  33,525.376 33,525.376      0.000
 Tail Gas      32.394     32.394   730.140  23,651.985 23,651.985      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,899.78  →  167,899.78   (Δ +0.00)
Feed cost    : 131,999.27 →  131,999.27  (Δ +0.00)
Recycle cred.: 14,109.20 →  14,109.20  (Δ +0.00)
ΔFG (tph)    : -0.116439 → -0.116439 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 50,082.12 → 50,082.12 (Δ -0.00)
[APPLIED next_stamp] 2025-01-04 09:00:00 -> 2025-01-04 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999275391, 'RCOT_chamber3': 852.9999999276855, 'RCOT_gas_chamber5': 889.9999999154297, 'RCOT_naphtha_chamber6': 852.9999998552246}
  rcots@next : {'RCOT_chamber2': 852.999999927832, 'RCOT_chamber3': 852.9999999279297, 'RCOT_gas_chamber5': 889.9999999150879, 'RCOT_naphtha_chamber6': 852.9999998557618}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-04 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.583     69.583   826.920  57,539.750 57,539.750      0.000
 Hydrogen       1.650      1.650 2,030.931   3,350.817  3,350.817      0.000
 Mixed C4      24.714     24.714   879.236  21,729.113 21,729.113      0.000
Propylene      36.032     36.032   805.800  29,034.190 29,034.190      0.000
      RPG      47.888     47.888   687.199  32,908.894 32,908.894     -0.000
 Tail Gas      32.504     32.504   730.140  23,732.184 23,732.184      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,294.95  →  168,294.95   (Δ +0.00)
Feed cost    : 131,557.00 →  131,557.00  (Δ +0.00)
Recycle cred.: 14,280.26 →  14,280.26  (Δ -0.00)
ΔFG (tph)    : -0.064611 → -0.064611 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 51,058.39 → 51,058.39 (Δ +0.00)
[APPLIED next_stamp] 2025-01-04 21:00:00 -> 2025-01-05 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.999999927832, 'RCOT_chamber3': 852.9999999279297, 'RCOT_gas_chamber5': 889.9999999150879, 'RCOT_naphtha_chamber6': 852.9999998557618}
  rcots@next : {'RCOT_chamber2': 852.9999999927734, 'RCOT_chamber3': 852.9999999927929, 'RCOT_gas_chamber5': 889.9999999915234, 'RCOT_naphtha_chamber6': 852.9999999855664}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-05 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.019     69.019   826.920  57,073.050 57,073.050      0.000
 Hydrogen       1.651      1.651 2,030.931   3,353.655  3,353.655      0.000
 Mixed C4      25.094     25.094   879.236  22,063.584 22,063.584      0.000
Propylene      36.199     36.199   805.800  29,169.416 29,169.416      0.000
      RPG      46.100     46.100   687.199  31,679.909 31,679.909      0.000
 Tail Gas      32.238     32.238   730.140  23,538.295 23,538.295      0.000

=== SUMMARY ($/h) ===
Revenue      : 166,877.91  →  166,877.91   (Δ +0.00)
Feed cost    : 132,081.90 →  132,081.90  (Δ +0.00)
Recycle cred.: 14,112.18 →  14,112.18  (Δ +0.00)
ΔFG (tph)    : -0.067446 → -0.067446 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 48,950.14 → 48,950.14 (Δ +0.00)
[APPLIED next_stamp] 2025-01-05 09:00:00 -> 2025-01-05 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927734, 'RCOT_chamber3': 852.9999999927929, 'RCOT_gas_chamber5': 889.9999999915234, 'RCOT_naphtha_chamber6': 852.9999999855664}
  rcots@next : {'RCOT_chamber2': 852.9999999927588, 'RCOT_chamber3': 852.999999992788, 'RCOT_gas_chamber5': 889.9999999915283, 'RCOT_naphtha_chamber6': 852.9999999855469}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-05 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      68.276     68.276   826.920  56,458.500 56,458.500      0.000
 Hydrogen       1.651      1.651 2,030.931   3,352.291  3,352.291      0.000
 Mixed C4      25.169     25.169   879.236  22,129.068 22,129.068      0.000
Propylene      36.101     36.101   805.800  29,090.101 29,090.101      0.000
      RPG      45.165     45.165   687.199  31,037.200 31,037.200      0.000
 Tail Gas      32.627     32.627   730.140  23,822.150 23,822.150      0.000

=== SUMMARY ($/h) ===
Revenue      : 165,889.31  →  165,889.31   (Δ +0.00)
Feed cost    : 132,943.04 →  132,943.04  (Δ +0.00)
Recycle cred.: 14,025.68 →  14,025.68  (Δ +0.00)
ΔFG (tph)    : -0.174198 → -0.174198 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 47,080.28 → 47,080.28 (Δ +0.00)
[APPLIED next_stamp] 2025-01-05 21:00:00 -> 2025-01-06 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927588, 'RCOT_chamber3': 852.999999992788, 'RCOT_gas_chamber5': 889.9999999915283, 'RCOT_naphtha_chamber6': 852.9999999855469}
  rcots@next : {'RCOT_chamber2': 852.9999999927197, 'RCOT_chamber3': 852.999999992749, 'RCOT_gas_chamber5': 889.9999999914209, 'RCOT_naphtha_chamber6': 852.9999999854688}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-06 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      68.964     68.964   826.920  57,027.302 57,027.302      0.000
 Hydrogen       1.666      1.666 2,030.931   3,383.899  3,383.899      0.000
 Mixed C4      25.401     25.401   879.236  22,333.797 22,333.797      0.000
Propylene      36.501     36.501   805.800  29,412.743 29,412.743      0.000
      RPG      43.350     43.350   687.199  29,789.806 29,789.806      0.000
 Tail Gas      31.939     31.939   730.140  23,320.172 23,320.172      0.000

=== SUMMARY ($/h) ===
Revenue      : 165,267.72  →  165,267.72   (Δ +0.00)
Feed cost    : 132,648.68 →  132,648.68  (Δ +0.00)
Recycle cred.: 14,013.28 →  14,013.28  (Δ +0.00)
ΔFG (tph)    : -0.136569 → -0.136569 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 46,717.26 → 46,717.26 (Δ +0.00)
[APPLIED next_stamp] 2025-01-06 09:00:00 -> 2025-01-06 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927197, 'RCOT_chamber3': 852.999999992749, 'RCOT_gas_chamber5': 889.9999999914209, 'RCOT_naphtha_chamber6': 852.9999999854688}
  rcots@next : {'RCOT_chamber2': 852.9999999926953, 'RCOT_chamber3': 852.9999999927197, 'RCOT_gas_chamber5': 889.9999999914551, 'RCOT_naphtha_chamber6': 852.999999985415}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-06 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.396     69.396   826.920  57,384.866 57,384.866      0.000
 Hydrogen       1.676      1.676 2,030.931   3,403.296  3,403.296      0.000
 Mixed C4      25.497     25.497   879.236  22,417.982 22,417.982      0.000
Propylene      36.329     36.329   805.800  29,273.729 29,273.729      0.000
      RPG      42.848     42.848   687.199  29,444.995 29,444.995      0.000
 Tail Gas      32.045     32.045   730.140  23,397.296 23,397.296      0.000

=== SUMMARY ($/h) ===
Revenue      : 165,322.16  →  165,322.16   (Δ +0.00)
Feed cost    : 131,296.21 →  131,296.21  (Δ +0.00)
Recycle cred.: 14,048.67 →  14,048.67  (Δ +0.00)
ΔFG (tph)    : -0.051173 → -0.051173 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 48,106.45 → 48,106.45 (Δ -0.00)
[APPLIED next_stamp] 2025-01-06 21:00:00 -> 2025-01-07 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999926953, 'RCOT_chamber3': 852.9999999927197, 'RCOT_gas_chamber5': 889.9999999914551, 'RCOT_naphtha_chamber6': 852.999999985415}
  rcots@next : {'RCOT_chamber2': 852.9999999927783, 'RCOT_chamber3': 852.9999999927734, 'RCOT_gas_chamber5': 889.9999999915332, 'RCOT_naphtha_chamber6': 852.9999999855518}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-07 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      67.956     67.956   826.920  56,194.557 56,194.557     -0.000
 Hydrogen       1.668      1.668 2,030.931   3,387.547  3,387.547     -0.000
 Mixed C4      25.661     25.661   879.236  22,561.853 22,561.853     -0.000
Propylene      36.324     36.324   805.800  29,269.842 29,269.842      0.000
      RPG      43.515     43.515   687.199  29,903.570 29,903.570      0.000
 Tail Gas      32.343     32.343   730.140  23,614.843 23,614.843     -0.000

=== SUMMARY ($/h) ===
Revenue      : 164,932.21  →  164,932.21   (Δ +0.00)
Feed cost    : 132,089.00 →  132,089.00  (Δ +0.00)
Recycle cred.: 14,038.45 →  14,038.45  (Δ -0.00)
ΔFG (tph)    : 6.402462 → 6.402462 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 42,899.84 → 42,899.84 (Δ +0.00)
[APPLIED next_stamp] 2025-01-07 09:00:00 -> 2025-01-07 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927783, 'RCOT_chamber3': 852.9999999927734, 'RCOT_gas_chamber5': 889.9999999915332, 'RCOT_naphtha_chamber6': 852.9999999855518}
  rcots@next : {'RCOT_chamber2': 852.9999999271485, 'RCOT_chamber3': 852.9999999271972, 'RCOT_gas_chamber5': 889.9999999915332, 'RCOT_naphtha_chamber6': 852.9999998543946}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          861.25 →  861.25  (Δ +0.00)
  RCOT_gas_chamber5          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-07 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      67.502     67.502   826.920  55,818.522 55,818.522      0.000
 Hydrogen       1.681      1.681 2,030.931   3,413.133  3,413.133      0.000
 Mixed C4      25.637     25.637   879.236  22,541.242 22,541.242     -0.000
Propylene      36.253     36.253   805.800  29,212.623 29,212.623     -0.000
      RPG      43.391     43.391   687.199  29,818.352 29,818.352     -0.000
 Tail Gas      32.841     32.841   730.140  23,978.812 23,978.812      0.000

=== SUMMARY ($/h) ===
Revenue      : 164,782.68  →  164,782.68   (Δ +0.00)
Feed cost    : 132,778.02 →  132,778.02  (Δ +0.00)
Recycle cred.: 13,958.01 →  13,958.01  (Δ +0.00)
ΔFG (tph)    : 6.367194 → 6.367194 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 42,002.79 → 42,002.79 (Δ +0.00)
[APPLIED next_stamp] 2025-01-07 21:00:00 -> 2025-01-08 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999271485, 'RCOT_chamber3': 852.9999999271972, 'RCOT_gas_chamber4': 861.2518016151315, 'RCOT_gas_chamber5': 889.9999999915332, 'RCOT_naphtha_chamber6': 852.9999998543946}
  rcots@next : {'RCOT_chamber2': 852.999999992705, 'RCOT_chamber3': 852.9999999927148, 'RCOT_gas_chamber4': 861.2518016151315, 'RCOT_gas_chamber5': 889.9999999915332, 'RCOT_naphtha_chamber6': 852.9999999854199}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 222.69
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          861.25 →  866.25  (Δ +5.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-08 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      67.888     68.489   826.920  56,137.546 56,634.893    497.347
 Hydrogen       1.662      1.715 2,030.931   3,375.620  3,482.362    106.742
 Mixed C4      25.562     25.663   879.236  22,475.418 22,563.457     88.039
Propylene      36.457     36.354   805.800  29,376.690 29,294.114    -82.576
      RPG      43.840     43.896   687.199  30,126.966 30,165.286     38.320
 Tail Gas      32.796     33.029   730.140  23,945.523 24,115.871    170.348

=== SUMMARY ($/h) ===
Revenue      : 165,437.76  →  166,255.98   (Δ +818.22)
Feed cost    : 133,183.37 →  133,183.37  (Δ +0.00)
Recycle cred.: 14,024.21 →  13,456.54  (Δ -567.66)
ΔFG (tph)    : -0.049303 → -0.004501 (Δ +0.044801) × Fuel Gas=621.92 → FG cost Δ: +27.86
MARGIN       : 46,309.26 → 46,531.95 (Δ +222.69)
[APPLIED next_stamp] 2025-01-08 09:00:00 -> 2025-01-08 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.999999992705, 'RCOT_chamber3': 852.9999999927148, 'RCOT_gas_chamber4': 861.2518016151315, 'RCOT_naphtha_chamber6': 852.9999999854199}
  rcots@next : {'RCOT_chamber2': 852.9999999927295, 'RCOT_chamber3': 852.9999999927295, 'RCOT_gas_chamber4': 866.2518016066696, 'RCOT_naphtha_chamber6': 852.999999985459}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 286.79
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          866.25 →  871.25  (Δ +5.00)
  RCOT_naphtha_chamber5      835.79 →  840.79  (Δ +5.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-08 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      68.008     68.788   826.920  56,237.404 56,882.340    644.936
 Hydrogen       1.748      1.810 2,030.931   3,551.050  3,676.540    125.489
 Mixed C4      25.580     25.672   879.236  22,490.955 22,571.558     80.602
Propylene      36.177     36.012   805.800  29,151.616 29,018.547   -133.069
      RPG      45.053     44.916   687.199  30,960.072 30,865.969    -94.103
 Tail Gas      32.589     32.953   730.140  23,794.331 24,060.464    266.134

=== SUMMARY ($/h) ===
Revenue      : 166,185.43  →  167,075.42   (Δ +889.99)
Feed cost    : 132,721.41 →  132,721.41  (Δ +0.00)
Recycle cred.: 13,632.51 →  13,067.89  (Δ -564.61)
ΔFG (tph)    : -0.052452 → 0.009601 (Δ +0.062053) × Fuel Gas=621.92 → FG cost Δ: +38.59
MARGIN       : 47,129.15 → 47,415.93 (Δ +286.79)
[APPLIED next_stamp] 2025-01-08 21:00:00 -> 2025-01-09 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927295, 'RCOT_chamber3': 852.9999999927295, 'RCOT_gas_chamber4': 866.2518016066696, 'RCOT_naphtha_chamber5': 835.7896851690493, 'RCOT_naphtha_chamber6': 852.999999985459}
  rcots@next : {'RCOT_chamber2': 852.9999999282714, 'RCOT_chamber3': 852.9999999283691, 'RCOT_gas_chamber4': 871.2518015181441, 'RCOT_naphtha_chamber5': 840.7896850810122, 'RCOT_naphtha_chamber6': 852.9999999446777}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 296.88
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          871.25 →  876.25  (Δ +5.00)
  RCOT_naphtha_chamber5      840.79 →  845.79  (Δ +5.00)
  RCOT_naphtha_chamber6      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-09 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.442     70.260   826.920  57,422.672 58,099.491    676.819
 Hydrogen       1.810      1.875 2,030.931   3,675.353  3,808.071    132.718
 Mixed C4      25.591     25.671   879.236  22,500.918 22,570.955     70.037
Propylene      35.870     35.664   805.800  28,904.024 28,737.742   -166.282
      RPG      45.402     45.182   687.199  31,200.269 31,049.362   -150.907
 Tail Gas      32.886     33.313   730.140  24,011.641 24,322.799    311.158

=== SUMMARY ($/h) ===
Revenue      : 167,714.88  →  168,588.42   (Δ +873.54)
Feed cost    : 131,652.14 →  131,652.14  (Δ +0.00)
Recycle cred.: 13,272.02 →  12,737.19  (Δ -534.83)
ΔFG (tph)    : 0.015429 → 0.082697 (Δ +0.067268) × Fuel Gas=621.92 → FG cost Δ: +41.84
MARGIN       : 49,325.16 → 49,622.04 (Δ +296.88)
[APPLIED next_stamp] 2025-01-09 09:00:00 -> 2025-01-09 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999282714, 'RCOT_chamber3': 852.9999999283691, 'RCOT_gas_chamber4': 871.2518015181441, 'RCOT_naphtha_chamber5': 840.7896850810122, 'RCOT_naphtha_chamber6': 852.9999999446777}
  rcots@next : {'RCOT_chamber2': 852.9999999928516, 'RCOT_chamber3': 852.9999999928565, 'RCOT_gas_chamber4': 876.2518015094186, 'RCOT_naphtha_chamber5': 845.7896850667202, 'RCOT_naphtha_chamber6': 853.0}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 258.13
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          876.25 →  881.25  (Δ +5.00)
  RCOT_naphtha_chamber5      845.79 →  850.79  (Δ +5.00)

=== PRICE SNAPSHOT @ 2025-01-09 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.910     70.649   826.920  57,809.661 58,421.145    611.484
 Hydrogen       1.882      1.945 2,030.931   3,821.198  3,950.580    129.382
 Mixed C4      25.433     25.509   879.236  22,361.884 22,428.130     66.247
Propylene      35.996     35.771   805.800  29,005.632 28,824.435   -181.197
      RPG      44.702     44.518   687.199  30,719.421 30,592.543   -126.878
 Tail Gas      32.746     33.164   730.140  23,909.346 24,214.089    304.743

=== SUMMARY ($/h) ===
Revenue      : 167,627.14  →  168,430.92   (Δ +803.78)
Feed cost    : 131,249.62 →  131,249.62  (Δ +0.00)
Recycle cred.: 12,676.73 →  12,168.94  (Δ -507.79)
ΔFG (tph)    : -0.005525 → 0.055351 (Δ +0.060876) × Fuel Gas=621.92 → FG cost Δ: +37.86
MARGIN       : 49,057.69 → 49,315.82 (Δ +258.13)
[APPLIED next_stamp] 2025-01-09 21:00:00 -> 2025-01-10 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928516, 'RCOT_chamber3': 852.9999999928565, 'RCOT_gas_chamber4': 876.2518015094186, 'RCOT_naphtha_chamber5': 845.7896850667202}
  rcots@next : {'RCOT_chamber2': 852.9999999999287, 'RCOT_chamber3': 852.9999999999288, 'RCOT_gas_chamber4': 881.2518015093308, 'RCOT_naphtha_chamber5': 850.7896850665777}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 172.77
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          881.25 →  886.25  (Δ +5.00)
  RCOT_naphtha_chamber5      850.79 →  853.00  (Δ +2.21)

=== PRICE SNAPSHOT @ 2025-01-10 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.213     70.689   826.920  58,060.261 58,454.317    394.056
 Hydrogen       1.940      1.993 2,030.931   3,940.069  4,048.103    108.034
 Mixed C4      25.580     25.685   879.236  22,490.937 22,582.934     91.997
Propylene      35.779     35.633   805.800  28,830.867 28,712.826   -118.041
      RPG      43.256     43.217   687.199  29,725.301 29,698.390    -26.910
 Tail Gas      33.224     33.518   730.140  24,258.220 24,473.025    214.804

=== SUMMARY ($/h) ===
Revenue      : 167,305.65  →  167,969.60   (Δ +663.94)
Feed cost    : 130,930.52 →  130,930.52  (Δ +0.00)
Recycle cred.: 12,201.38 →  11,734.62  (Δ -466.76)
ΔFG (tph)    : 0.118040 → 0.157297 (Δ +0.039257) × Fuel Gas=621.92 → FG cost Δ: +24.41
MARGIN       : 48,503.10 → 48,675.87 (Δ +172.77)
[APPLIED next_stamp] 2025-01-10 09:00:00 -> 2025-01-10 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999999287, 'RCOT_chamber3': 852.9999999999288, 'RCOT_gas_chamber4': 881.2518015093308, 'RCOT_naphtha_chamber5': 850.7896850665777}
  rcots@next : {'RCOT_chamber2': 852.9999999992986, 'RCOT_chamber3': 852.9999999992971, 'RCOT_gas_chamber4': 886.2518015084681, 'RCOT_naphtha_chamber5': 852.9999999985961}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 110.53
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          886.25 →  890.00  (Δ +3.75)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-10 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.574     70.806   826.920  58,359.052 58,550.683    191.631
 Hydrogen       1.974      2.010 2,030.931   4,008.835  4,081.845     73.010
 Mixed C4      25.192     25.291   879.236  22,149.406 22,236.411     87.005
Propylene      35.294     35.235   805.800  28,439.613 28,391.995    -47.618
      RPG      43.280     43.335   687.199  29,741.888 29,780.039     38.151
 Tail Gas      33.681     33.844   730.140  24,591.829 24,711.013    119.184

=== SUMMARY ($/h) ===
Revenue      : 167,290.62  →  167,751.99   (Δ +461.36)
Feed cost    : 134,415.27 →  134,415.27  (Δ +0.00)
Recycle cred.: 11,744.78 →  11,405.90  (Δ -338.88)
ΔFG (tph)    : -0.094094 → -0.074883 (Δ +0.019210) × Fuel Gas=621.92 → FG cost Δ: +11.95
MARGIN       : 44,678.66 → 44,789.19 (Δ +110.53)
[APPLIED next_stamp] 2025-01-10 21:00:00 -> 2025-01-11 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999992986, 'RCOT_chamber3': 852.9999999992971, 'RCOT_gas_chamber4': 886.2518015084681, 'RCOT_naphtha_chamber5': 852.9999999985961}
  rcots@next : {'RCOT_chamber2': 852.9999999927442, 'RCOT_chamber3': 852.9999999927442, 'RCOT_gas_chamber4': 889.9999999909815, 'RCOT_naphtha_chamber5': 852.9999999854883}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-11 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.484     70.484   826.920  58,284.608 58,284.608      0.000
 Hydrogen       2.009      2.009 2,030.931   4,080.192  4,080.192      0.000
 Mixed C4      25.247     25.247   879.236  22,198.383 22,198.383      0.000
Propylene      35.426     35.426   805.800  28,546.425 28,546.425      0.000
      RPG      43.043     43.043   687.199  29,579.004 29,579.004      0.000
 Tail Gas      33.896     33.896   730.140  24,748.646 24,748.646      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,437.26  →  167,437.26   (Δ +0.00)
Feed cost    : 133,624.79 →  133,624.79  (Δ +0.00)
Recycle cred.: 11,912.37 →  11,912.37  (Δ +0.00)
ΔFG (tph)    : -0.084099 → -0.084099 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 45,777.13 → 45,777.13 (Δ -0.00)
[APPLIED next_stamp] 2025-01-11 09:00:00 -> 2025-01-11 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927442, 'RCOT_chamber3': 852.9999999927442, 'RCOT_gas_chamber4': 889.9999999909815, 'RCOT_naphtha_chamber5': 852.9999999854883}
  rcots@next : {'RCOT_chamber2': 852.9999999927637, 'RCOT_chamber3': 852.9999999927783, 'RCOT_gas_chamber4': 889.9999999911084, 'RCOT_naphtha_chamber5': 852.999999985542}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-11 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.649     70.649   826.920  58,420.859 58,420.859      0.000
 Hydrogen       2.019      2.019 2,030.931   4,101.425  4,101.425      0.000
 Mixed C4      25.149     25.149   879.236  22,111.538 22,111.538      0.000
Propylene      35.534     35.534   805.800  28,633.015 28,633.015      0.000
      RPG      43.141     43.141   687.199  29,646.758 29,646.758      0.000
 Tail Gas      33.991     33.991   730.140  24,817.862 24,817.862      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,731.46  →  167,731.46   (Δ +0.00)
Feed cost    : 132,070.36 →  132,070.36  (Δ +0.00)
Recycle cred.: 11,827.88 →  11,827.88  (Δ +0.00)
ΔFG (tph)    : -0.011454 → -0.011454 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 47,496.10 → 47,496.10 (Δ -0.00)
[APPLIED next_stamp] 2025-01-11 21:00:00 -> 2025-01-12 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927637, 'RCOT_chamber3': 852.9999999927783, 'RCOT_gas_chamber4': 889.9999999911084, 'RCOT_naphtha_chamber5': 852.999999985542}
  rcots@next : {'RCOT_chamber2': 852.9999999928516, 'RCOT_chamber3': 852.9999999928663, 'RCOT_gas_chamber4': 889.9999999911084, 'RCOT_naphtha_chamber5': 852.9999999857226}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-12 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.545     70.545   826.920  58,335.286 58,335.286      0.000
 Hydrogen       1.935      1.935 2,030.931   3,930.353  3,930.353     -0.000
 Mixed C4      25.367     25.367   879.236  22,303.251 22,303.251      0.000
Propylene      35.402     35.402   805.800  28,526.564 28,526.564      0.000
      RPG      43.604     43.604   687.199  29,964.613 29,964.613     -0.000
 Tail Gas      33.726     33.726   730.140  24,624.850 24,624.850     -0.000

=== SUMMARY ($/h) ===
Revenue      : 167,684.92  →  167,684.92   (Δ +0.00)
Feed cost    : 132,585.53 →  132,585.53  (Δ +0.00)
Recycle cred.: 11,750.14 →  11,750.14  (Δ +0.00)
ΔFG (tph)    : -0.058686 → -0.058686 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 46,886.03 → 46,886.03 (Δ +0.00)
[APPLIED next_stamp] 2025-01-12 09:00:00 -> 2025-01-12 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928516, 'RCOT_chamber3': 852.9999999928663, 'RCOT_gas_chamber4': 889.9999999911084, 'RCOT_naphtha_chamber5': 852.9999999857226}
  rcots@next : {'RCOT_chamber2': 852.999999928125, 'RCOT_chamber3': 852.9999999282714, 'RCOT_gas_chamber4': 889.9999999111328, 'RCOT_naphtha_chamber5': 852.9999998563477}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-12 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.504     70.504   826.920  58,301.414 58,301.414      0.000
 Hydrogen       1.905      1.905 2,030.931   3,868.458  3,868.458      0.000
 Mixed C4      25.493     25.493   879.236  22,413.937 22,413.937     -0.000
Propylene      35.674     35.674   805.800  28,745.741 28,745.741     -0.000
      RPG      43.036     43.036   687.199  29,574.452 29,574.452     -0.000
 Tail Gas      33.891     33.891   730.140  24,745.443 24,745.443      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,649.44  →  167,649.44   (Δ +0.00)
Feed cost    : 133,444.48 →  133,444.48  (Δ +0.00)
Recycle cred.: 11,789.60 →  11,789.60  (Δ -0.00)
ΔFG (tph)    : -0.082953 → -0.082953 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 46,046.16 → 46,046.16 (Δ +0.00)
[APPLIED next_stamp] 2025-01-12 21:00:00 -> 2025-01-13 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.999999928125, 'RCOT_chamber3': 852.9999999282714, 'RCOT_gas_chamber4': 889.9999999111328, 'RCOT_naphtha_chamber5': 852.9999998563477}
  rcots@next : {'RCOT_chamber2': 852.9999999927637, 'RCOT_chamber3': 852.9999999927832, 'RCOT_gas_chamber4': 889.9999999910889, 'RCOT_naphtha_chamber5': 852.999999985542}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-13 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.229     69.229   826.920  57,246.543 57,246.543      0.000
 Hydrogen       1.847      1.847 2,030.931   3,750.271  3,750.271      0.000
 Mixed C4      25.577     25.577   879.236  22,487.781 22,487.781      0.000
Propylene      35.581     35.581   805.800  28,671.358 28,671.358      0.000
      RPG      44.771     44.771   687.199  30,766.358 30,766.358      0.000
 Tail Gas      34.139     34.139   730.140  24,926.436 24,926.436      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,848.75  →  167,848.75   (Δ +0.00)
Feed cost    : 132,472.43 →  132,472.43  (Δ +0.00)
Recycle cred.: 11,944.57 →  11,944.57  (Δ +0.00)
ΔFG (tph)    : -0.107537 → -0.107537 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 47,387.76 → 47,387.76 (Δ -0.00)
[APPLIED next_stamp] 2025-01-13 09:00:00 -> 2025-01-13 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927637, 'RCOT_chamber3': 852.9999999927832, 'RCOT_gas_chamber4': 889.9999999910889, 'RCOT_naphtha_chamber5': 852.999999985542}
  rcots@next : {'RCOT_chamber2': 852.9999999928125, 'RCOT_chamber3': 852.999999992832, 'RCOT_gas_chamber4': 889.9999999911474, 'RCOT_naphtha_chamber5': 852.9999999856396}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-13 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      68.780     68.780   826.920  56,875.620 56,875.620      0.000
 Hydrogen       1.818      1.818 2,030.931   3,691.271  3,691.271      0.000
 Mixed C4      25.580     25.580   879.236  22,490.725 22,490.725      0.000
Propylene      35.907     35.907   805.800  28,933.818 28,933.818      0.000
      RPG      45.751     45.751   687.199  31,439.785 31,439.785      0.000
 Tail Gas      33.985     33.985   730.140  24,814.093 24,814.093      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,245.31  →  168,245.31   (Δ +0.00)
Feed cost    : 131,893.29 →  131,893.29  (Δ +0.00)
Recycle cred.: 11,991.58 →  11,991.58  (Δ +0.00)
ΔFG (tph)    : -0.107376 → -0.107376 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 48,410.38 → 48,410.38 (Δ -0.00)
[APPLIED next_stamp] 2025-01-13 21:00:00 -> 2025-01-14 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928125, 'RCOT_chamber3': 852.999999992832, 'RCOT_gas_chamber4': 889.9999999911474, 'RCOT_naphtha_chamber5': 852.9999999856396}
  rcots@next : {'RCOT_chamber2': 852.9999999928174, 'RCOT_chamber3': 852.9999999928516, 'RCOT_gas_chamber4': 889.9999999911817, 'RCOT_naphtha_chamber5': 852.9999999856641}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-14 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      68.745     68.745   826.920  56,846.955 56,846.955      0.000
 Hydrogen       1.777      1.777 2,030.931   3,609.821  3,609.821      0.000
 Mixed C4      25.596     25.596   879.236  22,505.106 22,505.106      0.000
Propylene      35.763     35.763   805.800  28,818.074 28,818.074      0.000
      RPG      47.650     47.650   687.199  32,744.929 32,744.929      0.000
 Tail Gas      34.094     34.094   730.140  24,893.169 24,893.169      0.000

=== SUMMARY ($/h) ===
Revenue      : 169,418.05  →  169,418.05   (Δ +0.00)
Feed cost    : 133,370.27 →  133,370.27  (Δ +0.00)
Recycle cred.: 12,078.56 →  12,078.56  (Δ +0.00)
ΔFG (tph)    : -0.079648 → -0.079648 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 48,175.88 → 48,175.88 (Δ +0.00)
[APPLIED next_stamp] 2025-01-14 09:00:00 -> 2025-01-14 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928174, 'RCOT_chamber3': 852.9999999928516, 'RCOT_gas_chamber4': 889.9999999911817, 'RCOT_naphtha_chamber5': 852.9999999856641}
  rcots@next : {'RCOT_chamber2': 852.9999999927344, 'RCOT_chamber3': 852.9999999927637, 'RCOT_gas_chamber4': 889.9999999911768, 'RCOT_naphtha_chamber5': 852.999999985498}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-14 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      68.416     68.416   826.920  56,574.436 56,574.436      0.000
 Hydrogen       1.695      1.695 2,030.931   3,442.545  3,442.545      0.000
 Mixed C4      25.576     25.576   879.236  22,487.531 22,487.531      0.000
Propylene      35.824     35.824   805.800  28,866.592 28,866.592      0.000
      RPG      47.084     47.084   687.199  32,356.415 32,356.415      0.000
 Tail Gas      33.870     33.870   730.140  24,729.835 24,729.835      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,457.35  →  168,457.35   (Δ +0.00)
Feed cost    : 131,939.21 →  131,939.21  (Δ +0.00)
Recycle cred.: 12,196.54 →  12,196.54  (Δ +0.00)
ΔFG (tph)    : -0.035285 → -0.035285 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 48,736.62 → 48,736.62 (Δ -0.00)
[APPLIED next_stamp] 2025-01-14 21:00:00 -> 2025-01-15 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927344, 'RCOT_chamber3': 852.9999999927637, 'RCOT_gas_chamber4': 889.9999999911768, 'RCOT_naphtha_chamber5': 852.999999985498}
  rcots@next : {'RCOT_chamber2': 852.9999999928223, 'RCOT_chamber3': 852.9999999928369, 'RCOT_gas_chamber4': 889.9999999912402, 'RCOT_naphtha_chamber5': 852.9999999856592}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-15 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.024     69.024   826.920  57,077.121 57,077.121      0.000
 Hydrogen       1.540      1.540 2,030.931   3,127.905  3,127.905      0.000
 Mixed C4      25.517     25.517   879.236  22,435.535 22,435.535      0.000
Propylene      35.630     35.630   805.800  28,711.049 28,711.049      0.000
      RPG      45.504     45.504   687.199  31,270.180 31,270.180      0.000
 Tail Gas      33.821     33.821   730.140  24,694.250 24,694.250      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,316.04  →  167,316.04   (Δ +0.00)
Feed cost    : 131,165.91 →  131,165.91  (Δ +0.00)
Recycle cred.: 12,333.78 →  12,333.78  (Δ +0.00)
ΔFG (tph)    : -0.140939 → -0.140939 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 48,571.57 → 48,571.57 (Δ -0.00)
[APPLIED next_stamp] 2025-01-15 09:00:00 -> 2025-01-15 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928223, 'RCOT_chamber3': 852.9999999928369, 'RCOT_gas_chamber4': 889.9999999912402, 'RCOT_naphtha_chamber5': 852.9999999856592}
  rcots@next : {'RCOT_chamber2': 852.9999999928399, 'RCOT_chamber3': 852.9999999928399, 'RCOT_gas_chamber4': 889.9999999912042, 'RCOT_naphtha_chamber5': 852.9999999856798}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-15 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.023     70.023   826.920  57,903.483 57,903.483      0.000
 Hydrogen       1.515      1.515 2,030.931   3,075.854  3,075.854      0.000
 Mixed C4      25.547     25.547   879.236  22,461.695 22,461.695      0.000
Propylene      35.779     35.779   805.800  28,830.688 28,830.688      0.000
      RPG      43.971     43.971   687.199  30,217.076 30,217.076      0.000
 Tail Gas      33.896     33.896   730.140  24,749.122 24,749.122      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,237.92  →  167,237.92   (Δ +0.00)
Feed cost    : 132,014.57 →  132,014.57  (Δ +0.00)
Recycle cred.: 12,217.50 →  12,217.50  (Δ +0.00)
ΔFG (tph)    : -0.090305 → -0.090305 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 47,497.01 → 47,497.01 (Δ +0.00)
[APPLIED next_stamp] 2025-01-15 21:00:00 -> 2025-01-16 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928399, 'RCOT_chamber3': 852.9999999928399, 'RCOT_gas_chamber4': 889.9999999912042, 'RCOT_naphtha_chamber5': 852.9999999856798}
  rcots@next : {'RCOT_chamber2': 852.9999999928516, 'RCOT_chamber3': 852.9999999928272, 'RCOT_gas_chamber4': 889.9999999911817, 'RCOT_naphtha_chamber5': 852.9999999856788}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-16 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.047     70.047   826.920  57,922.958 57,922.958     -0.000
 Hydrogen       1.524      1.524 2,030.931   3,094.532  3,094.532     -0.000
 Mixed C4      25.553     25.553   879.236  22,466.698 22,466.698     -0.000
Propylene      35.680     35.680   805.800  28,750.643 28,750.643      0.000
      RPG      44.191     44.191   687.199  30,368.009 30,368.009      0.000
 Tail Gas      34.172     34.172   730.140  24,950.272 24,950.272     -0.000

=== SUMMARY ($/h) ===
Revenue      : 167,553.11  →  167,553.11   (Δ -0.00)
Feed cost    : 133,547.60 →  133,547.60  (Δ +0.00)
Recycle cred.: 12,322.38 →  12,322.38  (Δ +0.00)
ΔFG (tph)    : -0.119024 → -0.119024 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 46,401.91 → 46,401.91 (Δ +0.00)
[APPLIED next_stamp] 2025-01-16 09:00:00 -> 2025-01-16 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928516, 'RCOT_chamber3': 852.9999999928272, 'RCOT_gas_chamber4': 889.9999999911817, 'RCOT_naphtha_chamber5': 852.9999999856788}
  rcots@next : {'RCOT_chamber2': 852.9999999276855, 'RCOT_chamber3': 852.9999999277344, 'RCOT_gas_chamber4': 889.9999999105469, 'RCOT_naphtha_chamber5': 852.9999998554199}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-16 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.161     70.161   826.920  58,017.500 58,017.500      0.000
 Hydrogen       1.511      1.511 2,030.931   3,067.818  3,067.818      0.000
 Mixed C4      25.685     25.685   879.236  22,582.869 22,582.869      0.000
Propylene      35.764     35.764   805.800  28,818.325 28,818.325     -0.000
      RPG      44.404     44.404   687.199  30,514.277 30,514.277     -0.000
 Tail Gas      35.716     35.716   730.140  26,077.834 26,077.834      0.000

=== SUMMARY ($/h) ===
Revenue      : 169,078.62  →  169,078.62   (Δ +0.00)
Feed cost    : 133,338.35 →  133,338.35  (Δ +0.00)
Recycle cred.: 12,310.21 →  12,310.21  (Δ -0.00)
ΔFG (tph)    : -0.119366 → -0.119366 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 48,124.72 → 48,124.72 (Δ +0.00)
[APPLIED next_stamp] 2025-01-16 21:00:00 -> 2025-01-17 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999276855, 'RCOT_chamber3': 852.9999999277344, 'RCOT_gas_chamber4': 889.9999999105469, 'RCOT_naphtha_chamber5': 852.9999998554199}
  rcots@next : {'RCOT_chamber2': 852.9999999927637, 'RCOT_chamber3': 852.9999999927442, 'RCOT_gas_chamber4': 889.9999999910498, 'RCOT_naphtha_chamber5': 852.9999999855127}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-17 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.349     70.349   826.920  58,173.247 58,173.247      0.000
 Hydrogen       1.497      1.497 2,030.931   3,039.693  3,039.693      0.000
 Mixed C4      25.546     25.546   879.236  22,460.895 22,460.895      0.000
Propylene      35.590     35.590   805.800  28,678.300 28,678.300      0.000
      RPG      43.204     43.204   687.199  29,689.668 29,689.668      0.000
 Tail Gas      35.314     35.314   730.140  25,784.501 25,784.501      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,826.30  →  167,826.30   (Δ +0.00)
Feed cost    : 131,630.56 →  131,630.56  (Δ +0.00)
Recycle cred.: 12,094.60 →  12,094.60  (Δ +0.00)
ΔFG (tph)    : -0.055846 → -0.055846 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 48,325.07 → 48,325.07 (Δ -0.00)
[APPLIED next_stamp] 2025-01-17 09:00:00 -> 2025-01-17 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927637, 'RCOT_chamber3': 852.9999999927442, 'RCOT_gas_chamber4': 889.9999999910498, 'RCOT_naphtha_chamber5': 852.9999999855127}
  rcots@next : {'RCOT_chamber2': 852.9999999928614, 'RCOT_chamber3': 852.9999999928467, 'RCOT_gas_chamber4': 889.9999999912353, 'RCOT_naphtha_chamber5': 852.999999985708}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-17 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.020     70.020   826.920  57,900.938 57,900.938      0.000
 Hydrogen       1.507      1.507 2,030.931   3,060.272  3,060.272      0.000
 Mixed C4      25.430     25.430   879.236  22,358.630 22,358.630      0.000
Propylene      35.603     35.603   805.800  28,689.214 28,689.214      0.000
      RPG      43.180     43.180   687.199  29,673.521 29,673.521      0.000
 Tail Gas      35.418     35.418   730.140  25,860.197 25,860.197      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,542.77  →  167,542.77   (Δ +0.00)
Feed cost    : 132,066.67 →  132,066.67  (Δ +0.00)
Recycle cred.: 11,994.29 →  11,994.29  (Δ +0.00)
ΔFG (tph)    : -0.086949 → -0.086949 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 47,524.47 → 47,524.47 (Δ +0.00)
[APPLIED next_stamp] 2025-01-17 21:00:00 -> 2025-01-18 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928614, 'RCOT_chamber3': 852.9999999928467, 'RCOT_gas_chamber4': 889.9999999912353, 'RCOT_naphtha_chamber5': 852.999999985708}
  rcots@next : {'RCOT_chamber2': 852.9999999928614, 'RCOT_chamber3': 852.9999999928369, 'RCOT_gas_chamber4': 889.999999991211, 'RCOT_naphtha_chamber5': 852.9999999856982}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-18 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.251     70.251   826.920  58,091.720 58,091.720      0.000
 Hydrogen       1.497      1.497 2,030.931   3,040.219  3,040.219      0.000
 Mixed C4      25.567     25.567   879.236  22,479.126 22,479.126      0.000
Propylene      35.713     35.713   805.800  28,777.274 28,777.274      0.000
      RPG      43.670     43.670   687.199  30,009.673 30,009.673      0.000
 Tail Gas      35.470     35.470   730.140  25,897.807 25,897.807      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,295.82  →  168,295.82   (Δ +0.00)
Feed cost    : 132,744.52 →  132,744.52  (Δ +0.00)
Recycle cred.: 12,163.02 →  12,163.02  (Δ +0.00)
ΔFG (tph)    : -0.109890 → -0.109890 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 47,782.66 → 47,782.66 (Δ +0.00)
[APPLIED next_stamp] 2025-01-18 09:00:00 -> 2025-01-18 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928614, 'RCOT_chamber3': 852.9999999928369, 'RCOT_gas_chamber4': 889.999999991211, 'RCOT_naphtha_chamber5': 852.9999999856982}
  rcots@next : {'RCOT_chamber2': 852.9999999928174, 'RCOT_chamber3': 852.9999999927929, 'RCOT_gas_chamber4': 889.999999991123, 'RCOT_naphtha_chamber5': 852.9999999856103}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-18 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.153     70.153   826.920  58,010.789 58,010.789      0.000
 Hydrogen       1.500      1.500 2,030.931   3,046.156  3,046.156      0.000
 Mixed C4      25.402     25.402   879.236  22,334.721 22,334.721      0.000
Propylene      35.695     35.695   805.800  28,763.115 28,763.115      0.000
      RPG      44.088     44.088   687.199  30,297.443 30,297.443      0.000
 Tail Gas      35.306     35.306   730.140  25,778.235 25,778.235      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,230.46  →  168,230.46   (Δ +0.00)
Feed cost    : 132,332.33 →  132,332.33  (Δ +0.00)
Recycle cred.: 12,223.98 →  12,223.98  (Δ +0.00)
ΔFG (tph)    : -0.105031 → -0.105031 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 48,187.43 → 48,187.43 (Δ -0.00)
[APPLIED next_stamp] 2025-01-18 21:00:00 -> 2025-01-19 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928174, 'RCOT_chamber3': 852.9999999927929, 'RCOT_gas_chamber4': 889.999999991123, 'RCOT_naphtha_chamber5': 852.9999999856103}
  rcots@next : {'RCOT_chamber2': 852.9999999928174, 'RCOT_chamber3': 852.9999999928223, 'RCOT_gas_chamber4': 889.9999999911474, 'RCOT_naphtha_chamber5': 852.9999999856348}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-19 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.298     70.298   826.920  58,131.180 58,131.180      0.000
 Hydrogen       1.501      1.501 2,030.931   3,048.040  3,048.040      0.000
 Mixed C4      25.565     25.565   879.236  22,477.724 22,477.724      0.000
Propylene      35.640     35.640   805.800  28,718.438 28,718.438      0.000
      RPG      44.378     44.378   687.199  30,496.604 30,496.604      0.000
 Tail Gas      35.438     35.438   730.140  25,874.483 25,874.483      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,746.47  →  168,746.47   (Δ +0.00)
Feed cost    : 132,842.79 →  132,842.79  (Δ +0.00)
Recycle cred.: 12,291.12 →  12,291.12  (Δ +0.00)
ΔFG (tph)    : -0.080235 → -0.080235 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 48,244.70 → 48,244.70 (Δ +0.00)
[APPLIED next_stamp] 2025-01-19 09:00:00 -> 2025-01-19 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928174, 'RCOT_chamber3': 852.9999999928223, 'RCOT_gas_chamber4': 889.9999999911474, 'RCOT_naphtha_chamber5': 852.9999999856348}
  rcots@next : {'RCOT_chamber2': 852.9999999928027, 'RCOT_chamber3': 852.9999999928125, 'RCOT_gas_chamber4': 889.999999991123, 'RCOT_naphtha_chamber5': 852.9999999856152}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-19 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.107     70.107   826.920  57,972.535 57,972.535      0.000
 Hydrogen       1.504      1.504 2,030.931   3,054.599  3,054.599      0.000
 Mixed C4      25.658     25.658   879.236  22,559.280 22,559.280      0.000
Propylene      35.588     35.588   805.800  28,676.450 28,676.450      0.000
      RPG      44.557     44.557   687.199  30,619.263 30,619.263      0.000
 Tail Gas      35.315     35.315   730.140  25,785.177 25,785.177      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,667.31  →  168,667.31   (Δ +0.00)
Feed cost    : 134,175.89 →  134,175.89  (Δ +0.00)
Recycle cred.: 12,308.85 →  12,308.85  (Δ +0.00)
ΔFG (tph)    : -0.168981 → -0.168981 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 46,905.36 → 46,905.36 (Δ +0.00)
[APPLIED next_stamp] 2025-01-19 21:00:00 -> 2025-01-20 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999928027, 'RCOT_chamber3': 852.9999999928125, 'RCOT_gas_chamber4': 889.999999991123, 'RCOT_naphtha_chamber5': 852.9999999856152}
  rcots@next : {'RCOT_chamber2': 852.9999999927295, 'RCOT_chamber3': 852.9999999927295, 'RCOT_gas_chamber4': 889.9999999910157, 'RCOT_naphtha_chamber5': 852.9999999854639}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-20 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.234     70.234   826.920  58,078.166 58,078.166      0.000
 Hydrogen       1.501      1.501 2,030.931   3,048.179  3,048.179      0.000
 Mixed C4      25.660     25.660   879.236  22,560.973 22,560.973      0.000
Propylene      35.804     35.804   805.800  28,850.719 28,850.719      0.000
      RPG      44.520     44.520   687.199  30,594.398 30,594.398      0.000
 Tail Gas      35.229     35.229   730.140  25,722.290 25,722.290      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,854.72  →  168,854.72   (Δ +0.00)
Feed cost    : 133,142.49 →  133,142.49  (Δ +0.00)
Recycle cred.: 12,368.03 →  12,368.03  (Δ +0.00)
ΔFG (tph)    : -0.064611 → -0.064611 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 48,120.45 → 48,120.45 (Δ -0.00)
[APPLIED next_stamp] 2025-01-20 09:00:00 -> 2025-01-20 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927295, 'RCOT_chamber3': 852.9999999927295, 'RCOT_gas_chamber4': 889.9999999910157, 'RCOT_naphtha_chamber5': 852.9999999854639}
  rcots@next : {'RCOT_chamber2': 852.999999992788, 'RCOT_chamber3': 852.9999999927686, 'RCOT_gas_chamber4': 889.9999999910742, 'RCOT_naphtha_chamber5': 852.9999999855567}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-20 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.273     70.273   826.920  58,109.850 58,109.850      0.000
 Hydrogen       1.503      1.503 2,030.931   3,052.353  3,052.353      0.000
 Mixed C4      25.662     25.662   879.236  22,562.741 22,562.741      0.000
Propylene      35.884     35.884   805.800  28,915.404 28,915.404      0.000
      RPG      44.273     44.273   687.199  30,424.080 30,424.080      0.000
 Tail Gas      35.226     35.226   730.140  25,719.927 25,719.927      0.000

=== SUMMARY ($/h) ===
Revenue      : 168,784.35  →  168,784.35   (Δ +0.00)
Feed cost    : 133,276.22 →  133,276.22  (Δ +0.00)
Recycle cred.: 12,366.82 →  12,366.82  (Δ +0.00)
ΔFG (tph)    : -0.061041 → -0.061041 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 47,912.92 → 47,912.92 (Δ +0.00)
[APPLIED next_stamp] 2025-01-20 21:00:00 -> 2025-01-21 09:00:00
  rcots@ts   : {'RCOT_chamber2': 852.999999992788, 'RCOT_chamber3': 852.9999999927686, 'RCOT_gas_chamber4': 889.9999999910742, 'RCOT_naphtha_chamber5': 852.9999999855567}
  rcots@next : {'RCOT_chamber2': 852.9999999927783, 'RCOT_chamber3': 852.9999999927588, 'RCOT_gas_chamber4': 889.9999999910889, 'RCOT_naphtha_chamber5': 852.9999999855371}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-21 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.333     69.333   826.920  57,332.560 57,332.560      0.000
 Hydrogen       1.469      1.469 2,030.931   2,983.806  2,983.806      0.000
 Mixed C4      25.420     25.420   879.236  22,349.779 22,349.779      0.000
Propylene      35.579     35.579   805.800  28,669.160 28,669.160      0.000
      RPG      44.021     44.021   687.199  30,250.983 30,250.983      0.000
 Tail Gas      34.756     34.756   730.140  25,376.653 25,376.653      0.000

=== SUMMARY ($/h) ===
Revenue      : 166,962.94  →  166,962.94   (Δ +0.00)
Feed cost    : 132,454.41 →  132,454.41  (Δ +0.00)
Recycle cred.: 12,445.39 →  12,445.39  (Δ +0.00)
ΔFG (tph)    : 0.036432 → 0.036432 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 46,931.27 → 46,931.27 (Δ -0.00)
[APPLIED next_stamp] 2025-01-21 09:00:00 -> 2025-01-21 21:00:00
  rcots@ts   : {'RCOT_chamber2': 852.9999999927783, 'RCOT_chamber3': 852.9999999927588, 'RCOT_gas_chamber4': 889.9999999910889, 'RCOT_naphtha_chamber5': 852.9999999855371}
  rcots@next : {'RCOT_chamber2': 852.9999999928272, 'RCOT_chamber3': 852.9999999928027, 'RCOT_gas_chamber4': 889.9999999911279, 'RCOT_naphtha_chamber5': 852.999999985625}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 88.61
RCOT* (per chamber):
  RCOT_chamber1              828.73 →  833.73  (Δ +5.00)
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-21 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.445     69.717   826.920  57,425.556 57,650.749    225.193
 Hydrogen       1.388      1.397 2,030.931   2,818.486  2,838.075     19.589
 Mixed C4      25.589     25.588   879.236  22,498.652 22,497.825     -0.827
Propylene      35.744     35.748   805.800  28,802.439 28,805.944      3.505
      RPG      43.993     43.677   687.199  30,231.744 30,014.578   -217.166
 Tail Gas      34.680     34.814   730.140  25,321.145 25,418.803     97.657

=== SUMMARY ($/h) ===
Revenue      : 167,098.02  →  167,225.97   (Δ +127.95)
Feed cost    : 132,049.74 →  132,049.74  (Δ +0.00)
Recycle cred.: 12,474.32 →  12,481.51  (Δ +7.19)
ΔFG (tph)    : 0.142130 → 0.165077 (Δ +0.022946) × Fuel Gas=621.92 → FG cost Δ: +14.27
MARGIN       : 47,434.21 → 47,555.08 (Δ +120.87)
[APPLIED next_stamp] 2025-01-21 21:00:00 -> 2025-01-22 09:00:00
  rcots@ts   : {'RCOT_chamber1': 828.7285514296306, 'RCOT_chamber2': 852.9999999928272, 'RCOT_chamber3': 852.9999999928027, 'RCOT_gas_chamber4': 889.9999999911279, 'RCOT_naphtha_chamber5': 852.999999985625}
  rcots@next : {'RCOT_chamber1': 833.7285514296306, 'RCOT_chamber2': 853.0, 'RCOT_chamber3': 853.0, 'RCOT_gas_chamber4': 890.0, 'RCOT_naphtha_chamber5': 852.996926237085}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 171.04
RCOT* (per chamber):
  RCOT_chamber1              833.73 →  838.73  (Δ +5.00)
  RCOT_chamber2              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-22 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.960     70.389   826.920  57,851.514 58,205.878    354.365
 Hydrogen       1.417      1.432 2,030.931   2,877.151  2,908.866     31.715
 Mixed C4      25.613     25.598   879.236  22,520.059 22,506.565    -13.494
Propylene      35.810     35.786   805.800  28,855.471 28,836.189    -19.282
      RPG      43.276     42.800   687.199  29,739.273 29,412.366   -326.907
 Tail Gas      34.706     34.920   730.140  25,339.953 25,496.633    156.680

=== SUMMARY ($/h) ===
Revenue      : 167,183.42  →  167,366.50   (Δ +183.08)
Feed cost    : 132,678.14 →  132,678.14  (Δ +0.00)
Recycle cred.: 12,532.18 →  12,541.94  (Δ +9.77)
ΔFG (tph)    : 0.258718 → 0.293782 (Δ +0.035064) × Fuel Gas=621.92 → FG cost Δ: +21.81
MARGIN       : 46,876.55 → 47,047.59 (Δ +171.04)
[APPLIED next_stamp] 2025-01-22 09:00:00 -> 2025-01-22 21:00:00
  rcots@ts   : {'RCOT_chamber1': 833.7285514296306, 'RCOT_chamber2': 853.0, 'RCOT_gas_chamber4': 890.0, 'RCOT_naphtha_chamber5': 852.996926237085}
  rcots@next : {'RCOT_chamber1': 838.7285514223894, 'RCOT_chamber2': 852.9999999929101, 'RCOT_gas_chamber4': 889.9999999911377, 'RCOT_naphtha_chamber5': 852.999999985669}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 145.58
RCOT* (per chamber):
  RCOT_chamber1              838.73 →  843.73  (Δ +5.00)
  RCOT_chamber2              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-22 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.898     70.304   826.920  57,799.668 58,135.713    336.046
 Hydrogen       1.414      1.429 2,030.931   2,872.223  2,903.204     30.981
 Mixed C4      25.447     25.419   879.236  22,373.630 22,349.062    -24.568
Propylene      35.857     35.805   805.800  28,893.631 28,851.422    -42.208
      RPG      42.492     42.064   687.199  29,200.558 28,906.539   -294.019
 Tail Gas      34.954     35.161   730.140  25,520.934 25,672.744    151.810

=== SUMMARY ($/h) ===
Revenue      : 166,660.64  →  166,818.68   (Δ +158.04)
Feed cost    : 133,183.61 →  133,183.61  (Δ +0.00)
Recycle cred.: 12,465.70 →  12,473.28  (Δ +7.58)
ΔFG (tph)    : 0.221252 → 0.253470 (Δ +0.032218) × Fuel Gas=621.92 → FG cost Δ: +20.04
MARGIN       : 45,805.13 → 45,950.71 (Δ +145.58)
[APPLIED next_stamp] 2025-01-22 21:00:00 -> 2025-01-23 09:00:00
  rcots@ts   : {'RCOT_chamber1': 838.7285514223894, 'RCOT_chamber2': 852.9999999929101, 'RCOT_chamber3': 853.0, 'RCOT_gas_chamber4': 889.9999999911377, 'RCOT_naphtha_chamber5': 852.999999985669}
  rcots@next : {'RCOT_chamber1': 843.7285514151725, 'RCOT_chamber2': 852.9999999968652, 'RCOT_chamber3': 852.9999999959277, 'RCOT_gas_chamber4': 889.9999999911768, 'RCOT_naphtha_chamber5': 852.9999999855762}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 119.03
RCOT* (per chamber):
  RCOT_chamber1              843.73 →  848.73  (Δ +5.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-23 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.960     71.345   826.920  58,678.496 58,996.345    317.849
 Hydrogen       1.471      1.486 2,030.931   2,988.195  3,018.589     30.394
 Mixed C4      25.624     25.584   879.236  22,529.729 22,494.397    -35.331
Propylene      35.843     35.764   805.800  28,882.050 28,818.656    -63.394
      RPG      40.072     39.687   687.199  27,537.609 27,273.152   -264.457
 Tail Gas      35.114     35.315   730.140  25,637.973 25,784.843    146.870

=== SUMMARY ($/h) ===
Revenue      : 166,254.05  →  166,385.98   (Δ +131.93)
Feed cost    : 133,677.09 →  133,677.09  (Δ +0.00)
Recycle cred.: 12,288.41 →  12,293.87  (Δ +5.46)
ΔFG (tph)    : 0.026556 → 0.056070 (Δ +0.029514) × Fuel Gas=621.92 → FG cost Δ: +18.36
MARGIN       : 44,848.86 → 44,967.89 (Δ +119.03)
[APPLIED next_stamp] 2025-01-23 09:00:00 -> 2025-01-23 21:00:00
  rcots@ts   : {'RCOT_chamber1': 843.7285514151725, 'RCOT_chamber3': 852.9999999959277, 'RCOT_gas_chamber4': 889.9999999911768, 'RCOT_naphtha_chamber5': 852.9999999855762}
  rcots@next : {'RCOT_chamber1': 848.7285514079459, 'RCOT_chamber3': 852.999999992705, 'RCOT_gas_chamber4': 889.9999999911425, 'RCOT_naphtha_chamber5': 852.9999999854786}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 83.59
RCOT* (per chamber):
  RCOT_chamber1              848.73 →  853.00  (Δ +4.27)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-23 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      71.524     71.838   826.920  59,144.710 59,404.005    259.295
 Hydrogen       1.490      1.502 2,030.931   3,025.144  3,050.820     25.676
 Mixed C4      25.581     25.538   879.236  22,491.640 22,454.145    -37.495
Propylene      36.616     36.528   805.800  29,505.153 29,434.308    -70.845
      RPG      38.185     37.888   687.199  26,240.579 26,036.434   -204.145
 Tail Gas      35.456     35.624   730.140  25,888.033 26,010.808    122.774

=== SUMMARY ($/h) ===
Revenue      : 166,295.26  →  166,390.52   (Δ +95.26)
Feed cost    : 134,829.91 →  134,829.91  (Δ +0.00)
Recycle cred.: 12,205.01 →  12,207.82  (Δ +2.82)
ΔFG (tph)    : 0.053916 → 0.077209 (Δ +0.023293) × Fuel Gas=621.92 → FG cost Δ: +14.49
MARGIN       : 43,636.82 → 43,720.41 (Δ +83.59)
[APPLIED next_stamp] 2025-01-23 21:00:00 -> 2025-01-24 09:00:00
  rcots@ts   : {'RCOT_chamber1': 848.7285514079459, 'RCOT_chamber3': 852.999999992705, 'RCOT_gas_chamber4': 889.9999999911425, 'RCOT_naphtha_chamber5': 852.9999999854786}
  rcots@next : {'RCOT_chamber1': 852.9999999927197, 'RCOT_chamber3': 852.9999999926855, 'RCOT_gas_chamber4': 889.9999999910987, 'RCOT_naphtha_chamber5': 852.9999999854052}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber1              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-24 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      71.338     71.338   826.920  58,990.627 58,990.627      0.000
 Hydrogen       1.490      1.490 2,030.931   3,026.170  3,026.170      0.000
 Mixed C4      25.514     25.514   879.236  22,433.061 22,433.061      0.000
Propylene      36.715     36.715   805.800  29,584.789 29,584.789      0.000
      RPG      39.003     39.003   687.199  26,802.579 26,802.579      0.000
 Tail Gas      35.557     35.557   730.140  25,961.855 25,961.855      0.000

=== SUMMARY ($/h) ===
Revenue      : 166,799.08  →  166,799.08   (Δ +0.00)
Feed cost    : 134,258.64 →  134,258.64  (Δ +0.00)
Recycle cred.: 12,356.61 →  12,356.61  (Δ +0.00)
ΔFG (tph)    : 0.077862 → 0.077862 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 44,848.64 → 44,848.64 (Δ -0.00)
[APPLIED next_stamp] 2025-01-24 09:00:00 -> 2025-01-24 21:00:00
  rcots@ts   : {'RCOT_chamber1': 852.9999999927197, 'RCOT_chamber3': 852.9999999926855, 'RCOT_gas_chamber4': 889.9999999910987, 'RCOT_naphtha_chamber5': 852.9999999854052}
  rcots@next : {'RCOT_chamber1': 852.9999999927442, 'RCOT_chamber3': 852.999999992705, 'RCOT_gas_chamber4': 889.9999999911328, 'RCOT_naphtha_chamber5': 852.9999999854492}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber1              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-24 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      71.389     71.389   826.920  59,033.245 59,033.245      0.000
 Hydrogen       1.495      1.495 2,030.931   3,036.919  3,036.919     -0.000
 Mixed C4      25.792     25.792   879.236  22,677.607 22,677.607      0.000
Propylene      36.738     36.738   805.800  29,603.742 29,603.742      0.000
      RPG      38.311     38.311   687.199  26,327.561 26,327.561      0.000
 Tail Gas      35.891     35.891   730.140  26,205.565 26,205.565     -0.000

=== SUMMARY ($/h) ===
Revenue      : 166,884.64  →  166,884.64   (Δ +0.00)
Feed cost    : 134,177.84 →  134,177.84  (Δ +0.00)
Recycle cred.: 12,461.57 →  12,461.57  (Δ -0.00)
ΔFG (tph)    : 0.079606 → 0.079606 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 45,118.86 → 45,118.86 (Δ +0.00)
[APPLIED next_stamp] 2025-01-24 21:00:00 -> 2025-01-25 09:00:00
  rcots@ts   : {'RCOT_chamber1': 852.9999999927442, 'RCOT_chamber3': 852.999999992705, 'RCOT_gas_chamber4': 889.9999999911328, 'RCOT_naphtha_chamber5': 852.9999999854492}
  rcots@next : {'RCOT_chamber1': 852.9999999275391, 'RCOT_chamber3': 852.9999999270508, 'RCOT_gas_chamber4': 889.9999999110352, 'RCOT_naphtha_chamber5': 852.9999998545899}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber1              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-25 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      71.591     71.591   826.920  59,200.358 59,200.358      0.000
 Hydrogen       1.492      1.492 2,030.931   3,029.786  3,029.786      0.000
 Mixed C4      25.837     25.837   879.236  22,716.429 22,716.429     -0.000
Propylene      37.399     37.399   805.800  30,136.416 30,136.416     -0.000
      RPG      38.721     38.721   687.199  26,609.082 26,609.082     -0.000
 Tail Gas      35.794     35.794   730.140  26,134.815 26,134.815      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,826.89  →  167,826.89   (Δ +0.00)
Feed cost    : 133,367.58 →  133,367.58  (Δ +0.00)
Recycle cred.: 12,602.18 →  12,602.18  (Δ -0.00)
ΔFG (tph)    : 0.161190 → 0.161190 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 46,961.23 → 46,961.23 (Δ +0.00)
[APPLIED next_stamp] 2025-01-25 09:00:00 -> 2025-01-25 21:00:00
  rcots@ts   : {'RCOT_chamber1': 852.9999999275391, 'RCOT_chamber3': 852.9999999270508, 'RCOT_gas_chamber4': 889.9999999110352, 'RCOT_naphtha_chamber5': 852.9999998545899}
  rcots@next : {'RCOT_chamber1': 852.9999999926545, 'RCOT_chamber3': 852.9999999926097, 'RCOT_gas_chamber4': 889.9999999909762, 'RCOT_naphtha_chamber5': 852.9999999852641}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber1              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-25 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      71.433     71.433   826.920  59,069.317 59,069.317      0.000
 Hydrogen       1.496      1.496 2,030.931   3,039.174  3,039.174      0.000
 Mixed C4      25.811     25.811   879.236  22,694.032 22,694.032      0.000
Propylene      37.555     37.555   805.800  30,261.811 30,261.811      0.000
      RPG      38.953     38.953   687.199  26,768.653 26,768.653      0.000
 Tail Gas      35.763     35.763   730.140  26,111.728 26,111.728      0.000

=== SUMMARY ($/h) ===
Revenue      : 167,944.71  →  167,944.71   (Δ +0.00)
Feed cost    : 133,696.32 →  133,696.32  (Δ +0.00)
Recycle cred.: 12,725.70 →  12,725.70  (Δ +0.00)
ΔFG (tph)    : 0.123026 → 0.123026 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 46,897.59 → 46,897.59 (Δ +0.00)
[APPLIED next_stamp] 2025-01-25 21:00:00 -> 2025-01-26 09:00:00
  rcots@ts   : {'RCOT_chamber1': 852.9999999926545, 'RCOT_chamber3': 852.9999999926097, 'RCOT_gas_chamber4': 889.9999999909762, 'RCOT_naphtha_chamber5': 852.9999999852641}
  rcots@next : {'RCOT_chamber1': 852.9999999886862, 'RCOT_chamber3': 852.9999999886023, 'RCOT_gas_chamber4': 889.9999999861703, 'RCOT_naphtha_chamber5': 852.9999999772886}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber1              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-26 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      70.771     70.771   826.920  58,522.145 58,522.145      0.000
 Hydrogen       1.499      1.499 2,030.931   3,043.353  3,043.353      0.000
 Mixed C4      25.730     25.730   879.236  22,622.807 22,622.807      0.000
Propylene      37.174     37.174   805.800  29,954.463 29,954.463      0.000
      RPG      38.343     38.343   687.199  26,349.139 26,349.139      0.000
 Tail Gas      35.752     35.752   730.140  26,104.055 26,104.055      0.000

=== SUMMARY ($/h) ===
Revenue      : 166,595.96  →  166,595.96   (Δ +0.00)
Feed cost    : 132,609.08 →  132,609.08  (Δ +0.00)
Recycle cred.: 12,701.46 →  12,701.46  (Δ +0.00)
ΔFG (tph)    : 0.110830 → 0.110830 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 46,619.41 → 46,619.41 (Δ -0.00)
[APPLIED next_stamp] 2025-01-26 09:00:00 -> 2025-01-26 21:00:00
  rcots@ts   : {'RCOT_chamber1': 852.9999999886862, 'RCOT_chamber3': 852.9999999886023, 'RCOT_gas_chamber4': 889.9999999861703, 'RCOT_naphtha_chamber5': 852.9999999772886}
  rcots@next : {'RCOT_chamber1': 852.9999999928174, 'RCOT_chamber3': 852.9999999927588, 'RCOT_gas_chamber4': 889.9999999912061, 'RCOT_naphtha_chamber5': 852.9999999855762}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber1              853.00 →  853.00  (Δ -0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-26 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.847     69.847   826.920  57,758.137 57,758.137      0.000
 Hydrogen       1.488      1.488 2,030.931   3,021.072  3,021.072      0.000
 Mixed C4      25.686     25.686   879.236  22,583.717 22,583.717      0.000
Propylene      36.775     36.775   805.800  29,633.283 29,633.283      0.000
      RPG      38.496     38.496   687.199  26,454.246 26,454.246      0.000
 Tail Gas      35.828     35.828   730.140  26,159.413 26,159.413      0.000

=== SUMMARY ($/h) ===
Revenue      : 165,609.87  →  165,609.87   (Δ +0.00)
Feed cost    : 134,537.84 →  134,537.84  (Δ +0.00)
Recycle cred.: 12,639.58 →  12,639.58  (Δ +0.00)
ΔFG (tph)    : -0.058299 → -0.058299 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 43,747.87 → 43,747.87 (Δ +0.00)
[APPLIED next_stamp] 2025-01-26 21:00:00 -> 2025-01-27 09:00:00
  rcots@ts   : {'RCOT_chamber1': 852.9999999928174, 'RCOT_chamber3': 852.9999999927588, 'RCOT_gas_chamber4': 889.9999999912061, 'RCOT_naphtha_chamber5': 852.9999999855762}
  rcots@next : {'RCOT_chamber1': 852.9999999926855, 'RCOT_chamber3': 852.9999999926855, 'RCOT_gas_chamber4': 889.9999999910987, 'RCOT_naphtha_chamber5': 852.9999999853711}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: -0.00
RCOT* (per chamber):
  RCOT_chamber1              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ +0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ +0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ +0.00)

=== PRICE SNAPSHOT @ 2025-01-27 09:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.474     69.474   826.920  57,449.562 57,449.562      0.000
 Hydrogen       1.466      1.466 2,030.931   2,977.651  2,977.651      0.000
 Mixed C4      25.773     25.773   879.236  22,660.815 22,660.815      0.000
Propylene      35.969     35.969   805.800  28,983.691 28,983.691      0.000
      RPG      38.485     38.485   687.199  26,446.781 26,446.781      0.000
 Tail Gas      35.696     35.696   730.140  26,062.954 26,062.954      0.000

=== SUMMARY ($/h) ===
Revenue      : 164,581.45  →  164,581.45   (Δ +0.00)
Feed cost    : 134,092.28 →  134,092.28  (Δ +0.00)
Recycle cred.: 12,724.88 →  12,724.88  (Δ +0.00)
ΔFG (tph)    : -0.067735 → -0.067735 (Δ +0.000000) × Fuel Gas=621.92 → FG cost Δ: +0.00
MARGIN       : 43,256.17 → 43,256.17 (Δ -0.00)
[APPLIED next_stamp] 2025-01-27 09:00:00 -> 2025-01-27 21:00:00
  rcots@ts   : {'RCOT_chamber1': 852.9999999926855, 'RCOT_chamber3': 852.9999999926855, 'RCOT_gas_chamber4': 889.9999999910987, 'RCOT_naphtha_chamber5': 852.9999999853711}
  rcots@next : {'RCOT_chamber1': 852.9999999927148, 'RCOT_chamber3': 852.999999992705, 'RCOT_gas_chamber4': 889.9999999911621, 'RCOT_naphtha_chamber5': 852.9999999854199}

=== MULTI-KNOB RESULT ===
Status: ok
ΔMargin $/h: 0.00
RCOT* (per chamber):
  RCOT_chamber1              853.00 →  853.00  (Δ +0.00)
  RCOT_chamber3              853.00 →  853.00  (Δ -0.00)
  RCOT_gas_chamber4          890.00 →  890.00  (Δ -0.00)
  RCOT_naphtha_chamber5      853.00 →  853.00  (Δ -0.00)

=== PRICE SNAPSHOT @ 2025-01-27 21:00:00 ===
{'Ethylene': 826.9195557, 'Propylene': 805.7999878, 'Mixed C4': 879.2357788, 'RPG': 687.1991577, 'Hydrogen': 2030.930786, 'Tail Gas': 730.1395264, 'Fuel Gas': 621.9199829, 'PN': 676.8546753, 'Gas Feed': 0.0, 'LPG': 567.9401855, 'MX Offgas': 683.4105835}

=== BEFORE vs AFTER (per product) ===
  product  qty_before  qty_after     price  rev_before  rev_after  rev_delta
 Ethylene      69.982     69.982   826.920  57,869.180 57,869.180      0.000
 Hydrogen       1.463      1.463 2,030.931   2,970.950  2,970.950      0.000
 Mixed C4      25.808     25.808   879.236  22,691.317 22,691.317      0.000
Propylene      35.927     35.927   805.800  28,950.184 28,950.184      0.000
      RPG      38.460     38.460   687.199  26,429.982 26,429.982      0.000
 Tail Gas      35.725     35.725   730.140  26,083.886 26,083.886      0.000

=== SUMMARY ($/h) ===
Revenue      : 164,995.50  →  164,995.50   (Δ +0.00)
Feed cost    : 134,033.01 →  134,033.01  (Δ +0.00)
Recycle cred.: 12,806.86 →  12,806.86  (Δ +0.00)
ΔFG (tph)    : -0.056859 → -0.056859 (Δ -0.000000) × Fuel Gas=621.92 → FG cost Δ: -0.00
MARGIN       : 43,804.70 → 43,804.70 (Δ +0.00)
[APPLIED next_stamp] 2025-01-27 21:00:00 -> 2025-01-28 09:00:00
  rcots@ts   : {'RCOT_chamber1': 852.9999999927148, 'RCOT_chamber3': 852.999999992705, 'RCOT_gas_chamber4': 889.9999999911621, 'RCOT_naphtha_chamber5': 852.9999999854199}
  rcots@next : {'RCOT_chamber1': 852.9999999927295, 'RCOT_chamber3': 852.9999999926514, 'RCOT_gas_chamber4': 889.9999999911036, 'RCOT_naphtha_chamber5': 852.9999999853809}

In [ ]:
X_12h[X_12h.index=='2025-01-01']

In [ ]:
449.64/567.9401855

In [ ]:
# From your run: 
# • ΔFG (burner) = +0.125913 t/h • fuel_gas_heat_content_kcal_per_ton = 15_294_088 kcal/t (= 15.294 Gcal/t)
# Your utility shows 427.882 MMkcal/h on 2025‑01‑01. So the modeled Δ is ~0.45% of the meter (1.926 / 427.882 ≈ 0.0045). 
# That’s a perfectly reasonable order of magnitude for +5 °C on a handful of RCOT knobs with ~220 t/h fresh feed.


# normalize the meter 
#  (1.926 / 427.882 ≈ 0.0045, or 0.45%)
# and that is 
# delta Q = how much stuff flows (m mix) * Cp,mix (how hard it is to heat) * delta T (how much temperature you add) / eta heater (losses up the stack)

# Plugging in simple, round numbers
# Use a middle-of-the-road case:

# 𝑚 mix=300 t/h=300,000 kg/h
# 𝐶𝑝=5 kJ/kg-K
# =5 kJ/kg-K
# Δ𝑇=5 K
# 𝜂=0.90
# Compute the heat per hour:
# 300,000 kg/h×5 kJ/kg-K×5 K=7,500,000 kJ/h
# =7,500,000 kJ/h

# Divide by efficiency: 
# 7,500,000/0.90
# =8,333,333 kJ/h
# 7,500,000/0.90=8,333,333 kJ/h
# Convert to Gcal/h (1 Gcal = 4,184,000 kJ):
# 8,333,333/4,184,000
# = 1.99 Gcal/h
# 8,333,333/4,184,000=1.99 Gcal/h

# So a +5 °C step costs ≈2.0 Gcal/h in this “middle” case.
# What range should you expect?
# If you vary the inputs within realistic bounds:

# 𝑚 mix=300–320 t/h
# 𝐶𝑝=4–6 kJ/kg-K
# 𝜂=0.85–0.92
# Δ𝑇=5 °C

# =4–6 kJ/kg-K

# 𝜂=0.85–0.92
# Δ𝑇=5 °C

# 𝑇=5 °C
# ΔT=5 °C

# You get ~1.6 to 2.6 Gcal/h.
# Your modeled 1.926 Gcal/h lands neatly inside that band → it’s physically reasonable.
# Converting that heat to extra fuel-gas mass
# You already did this correctly: divide by the gas HHV.
# HHV ≈ 15.294 Gcal/t
# Extra heat ≈ 1.926 Gcal/h
# Extra FG ≈ 1.926/15.294=0.126
#  t/h
# 1.926/15.294=0.126 t/h

# Two handy “rules of thumb”

# Per degree, per flow:
# With 𝐶𝑝≈5 kJ/kg-K and 𝜂≈0.90, each 1 °C needs about
# 0.00133 Gcal/h for every 1 t/h flowing through the coils.
# So for 300 t/h, +5 °C ≈ 300×5×0.00133
# ≈2.0 Gcal/h
# 300×5×0.00133≈2.0 Gcal/h.

# Fuel mass from heat:
# Extra FG (t/h) ≈ Δ𝑄
# ΔQ (Gcal/h) ÷ HHV (Gcal/t).
# With HHV = 15.294, 2.0 Gcal/h → ~0.13 t/h.

# Tiny nuance (don’t overthink it)